**Histórico dos dados dos fundos de investimentos**
* https://dados.cvm.gov.br/dataset/fi-doc-cda
* https://dados.cvm.gov.br/dataset/fi-doc-inf_diario

* Observação: muitas vezes, os arquivos excel do site acima não se atualizam rapidamente. Para ver a posição mais atualizada entrar no site:
* https://conteudo.cvm.gov.br/menu/regulados/fundos/consultas/fundos.html
---
O arquivo zipado 'cda_fi' contém vários arquivos excel. Os arquivos que eu tenho interesse são:

* cda_fi_BLC_1: arquivo que contém os 'Títulos Públicos';
* cda_fi_BLC_2: arquivo que contém as 'Cotas de Fundos';
* cda_fi_BLC_4: arquivo que contém as 'Ações', 'BDRs', 'Opções', 'Mercado futuro', 'Debêntures', 'Obrigações por ações' (posição vendida);
* cda_fi_BLC_7: arquivo que contém os 'Investimentos no Exterior';
* cda_fi_BLC_8: arquivo que contém os 'Investimentos no Exterior' e 'Units';
* cda_fi_PL: arquivo que contém o PL dos fundos de investimentos.

---
O arquivo zipado 'inf_diario_fi' contém apenas um arquivo excel.

In [1]:
from funcoes import *
import glob
import os
import re
import zipfile

## Extraindo os arquivos zipados

### Extração dos arquivos "cda_fi"

In [2]:
# Mês e ano escolhido
mes = '02'
ano = '2025'

# Nome do arquivo zipado
zip_filename = f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm_zip//cda_fi_{ano}{mes}.zip'

# Lista dos nomes dos arquivos que você deseja extrair
arquivos_a_extrair = [
    f'cda_fi_BLC_1_{ano}{mes}.csv', 
    f'cda_fi_BLC_2_{ano}{mes}.csv', 
    f'cda_fi_BLC_4_{ano}{mes}.csv', 
    f'cda_fi_BLC_7_{ano}{mes}.csv', 
    f'cda_fi_BLC_8_{ano}{mes}.csv',
    f'cda_fi_PL_{ano}{mes}.csv'
]

# Caminho do diretório do arquivo não zipado
destino_dir = f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}'

# Abrindo o arquivo zipado
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    for arquivo in arquivos_a_extrair:
        zip_ref.extract(arquivo, destino_dir)

        csv_path = f'{destino_dir}//{arquivo}'
        parquet_path = f'{destino_dir}//{arquivo.replace(".csv", ".parquet")}'

        # Tentativa de leitura com fallback
        try:
            df = pd.read_csv(
                csv_path, sep=';', encoding='ISO-8859-1',
                low_memory=False, engine='c'
            )
        except Exception as e:
            print(f"[AVISO] Problema ao ler {arquivo} com engine='c': {e}")
            print("→ Tentando novamente com engine='python' e on_bad_lines='warn'")
            df = pd.read_csv(
                csv_path, sep=';', encoding='ISO-8859-1',
                engine='python', on_bad_lines='warn'
            )

        # Salvando em parquet
        df.to_parquet(parquet_path)

# Entrando no diretório especificado
os.chdir(destino_dir)

# Encontrando todos os arquivos .csv no diretório
arquivos_csv = glob.glob('*.csv')

for arquivo in arquivos_csv:
    os.remove(arquivo)

In [ ]:
# Anos mais antigos (até 2022), o arquivo 'cda_fi' é anual

# Ano escolhido
ano = '2019'

# Nome do arquivo zipado
zip_filename = f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm_zip//cda_fi_{ano}.zip'

# # Visualizando a lista dos nomes dos arquivos dentro do arquivo zipado
# with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
#     print("Arquivos dentro do arquivo zipado:")
#     print(zip_ref.namelist())

# Lista dos nomes dos arquivos que você deseja extrair
arquivos_a_extrair = [
    f'cda_fi_BLC_1_{ano}.csv', 
    f'cda_fi_BLC_2_{ano}.csv', 
    f'cda_fi_BLC_4_{ano}.csv', 
    f'cda_fi_BLC_7_{ano}.csv', 
    f'cda_fi_BLC_8_{ano}.csv',
    f'cda_fi_PL_{ano}.csv'
]

# Caminho do diretório do diretório do arquivo não zipado
destino_dir = f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}'

# Abrindo o arquivo zipado
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    # Extraindo os arquivos selecionados para um diretório específico
    for arquivo in arquivos_a_extrair:
        zip_ref.extract(arquivo, destino_dir)

        # Caminhos dos arquivos em formato csv e parquet
        csv_path = f'{destino_dir}//{arquivo}'
        parquet_path = f'{destino_dir}//{arquivo.replace(".csv", ".parquet")}'

        # Lendo os arquivos csv
        df = pd.read_csv(csv_path, sep=';', encoding='ISO-8859-1', low_memory=False)

        # Transformando esses arquivos em parquet
        df.to_parquet(parquet_path)

# Entrando para o diretório especificado
os.chdir(destino_dir)

# Encontrando todos os arquivos .csv no diretório
arquivos_csv = glob.glob('*.csv')

# Loop para deletar cada arquivo csv encontrado
for arquivo in arquivos_csv:
    os.remove(arquivo)

### Extração dos arquivos "inf_diario_fi"

In [ ]:
# Descompactando apenas um arquivo

# Mês e ano escolhido
mes = '12'
ano = '2024'

# Nome do arquivo zipado
zip_filename = f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//inf_diario_cvm_zip//inf_diario_fi_{ano}{mes}.zip'

# Lista dos nomes dos arquivos que você deseja extrair
arquivo_a_extrair = [f'inf_diario_fi_{ano}{mes}.csv']

# Caminho do diretório do diretório do arquivo não zipado
destino_dir = f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//inf_diario_cvm'

# Abrindo o arquivo zipado
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    # Extraindo os arquivos selecionados para um diretório específico
    for arquivo in arquivo_a_extrair:
        zip_ref.extract(arquivo, destino_dir)

        # Caminhos dos arquivos em formato csv e parquet
        csv_path = f'{destino_dir}//{arquivo}'
        parquet_path = f'{destino_dir}//{arquivo.replace(".csv", ".parquet")}'

        # Lendo os arquivos csv
        df = pd.read_csv(csv_path, sep=';', encoding='ISO-8859-1', low_memory=False)

        # Transformando esses arquivos em parquet
        df.to_parquet(parquet_path)

# Entrando para o diretório especificado
os.chdir(destino_dir)

# Encontrando todos os arquivos .csv no diretório
arquivos_csv = glob.glob('*.csv')

# Loop para deletar cada arquivo csv encontrado
for arquivo in arquivos_csv:
    os.remove(arquivo)

In [ ]:
# Descompactando vários arquivos
for _ in ['02', '03', '04', '05']: 
    
    # Ano escolhido
    ano = '2025'

    # Nome do arquivo zipado
    zip_filename = f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//inf_diario_cvm_zip//inf_diario_fi_{ano}{_}.zip'

    # Lista dos nomes dos arquivos que você deseja extrair
    arquivo_a_extrair = [f'inf_diario_fi_{ano}{_}.csv']

    # Caminho do diretório do diretório do arquivo não zipado
    destino_dir = f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//inf_diario_cvm'

    # Abrindo o arquivo zipado
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        # Extraindo os arquivos selecionados para um diretório específico
        for arquivo in arquivo_a_extrair:
            zip_ref.extract(arquivo, destino_dir)

            # Caminhos dos arquivos em formato csv e parquet
            csv_path = f'{destino_dir}//{arquivo}'
            parquet_path = f'{destino_dir}//{arquivo.replace(".csv", ".parquet")}'

            # Lendo os arquivos csv
            df = pd.read_csv(csv_path, sep=';', encoding='ISO-8859-1', low_memory=False)

            # Transformando esses arquivos em parquet
            df.to_parquet(parquet_path)

    # Entrando para o diretório especificado
    os.chdir(destino_dir)

    # Encontrando todos os arquivos .csv no diretório
    arquivos_csv = glob.glob('*.csv')

    # Loop para deletar cada arquivo csv encontrado
    for arquivo in arquivos_csv:
        os.remove(arquivo)

## Portfólios dos fundos de investimentos 

In [3]:
# Mês e ano escolhido
mes = '02'
ano = '2025'

In [4]:
# Formatando os arquivos parquet 'cda_fi'
df_cda_1 = open_cda_1(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_BLC_1_{ano}{mes}.parquet'
)

df_cda_2 = open_cda_2(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_BLC_2_{ano}{mes}.parquet'
)

df_cda_4 = open_cda_4(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_BLC_4_{ano}{mes}.parquet'
)

df_cda_7 = open_cda_7(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_BLC_7_{ano}{mes}.parquet'
)

df_cda_8 = open_cda_8(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_BLC_8_{ano}{mes}.parquet'
)

# Concatenando os dfs
df_ativos = pd.concat([df_cda_1, df_cda_2, df_cda_4, df_cda_7, df_cda_8])
df_ativos.tail()

,TP_FUNDO,CNPJ_FUNDO,DENOM_SOCIAL,DT_COMPTC,TP_APLIC,TP_ATIVO,VL_MERC_POS_FINAL,CD_ATIVO
229019,FI,41.283.527/0001-58,SICREDI - FIF EM AÇÕES SUSTENTÁVEIS ESG ALOCAÇ...,2025-02-28,Ações,Ação ordinária,0.0,SRNA/SRNA3/BRSRNAACNOR4
203605,FI,28.065.429/0001-86,RENDA LIQUIDEZ FIM CRÉDITO PRIVADO - INVESTIME...,2025-02-28,Ações,Ação ordinária,0.0,Ações - RCSL3
153637,FI,10.199.934/0001-58,BANRISUL DIVIDENDOS FUNDO DE INVESTIMENTO FINA...,2025-02-28,Ações,Ação ordinária,0.0,DIR SUB. ITAU
219344,FI,36.352.662/0001-78,ARX INCOME MASTER FIF - CLASSE DE INVESTIMENTO...,2025-02-28,Ações,Ação preferencial,0.0,LOA/LOAR4/BRLOARACNPR4
228363,FI,41.036.284/0001-53,DAEMON NOUS GLOBAL MASTER UNDO DE INVESTIMENTO...,2025-02-28,Ações,Ação ordinária,0.0,MMXM/MMXM3/BRMMXMACNOR2


In [ ]:
# Quais são os tipos de ativos o fundo de investimento possui
lista_ativos = df_ativos['TP_APLIC'].drop_duplicates().to_list()
lista_ativos

In [5]:
# Formatando os arquivos parquet 'inf_diario_fi'

# Caminho da pasta dos arquivos 'inf_diario_fi'
path_inf_diario = 'C:\\Users\\vitor\\projetos_python\\python_b3\\historico-arquivos\\fundos_investimentos\\inf_diario_cvm\\'
# Caminho completo de todos os arquivos que estão na pasta
paths_arquivos = glob.glob(path_inf_diario + '*.parquet')

# Selecionando os arquivos de 'inf_diario_fi_202212' até o 'inf_diario_fi_202407'
paths_arquivos_filt = [path for path in paths_arquivos if '202212' <= path[-14:-8] <= '202407']
# Lendo os arquivos parquet e juntando em apenas um df
lst_df = [pd.read_parquet(path) for path in paths_arquivos_filt]
# Concatenando os dfs
df_inf_diario = pd.concat(lst_df, ignore_index=True)
# Selecionando os 'Fundos de Investimentos' 
filt_fi = (df_inf_diario['TP_FUNDO'] == 'FI') 
df_inf_diario = df_inf_diario.loc[filt_fi]
# Selecionando as principais colunas
df_inf_diario = df_inf_diario[['TP_FUNDO', 'CNPJ_FUNDO', 'DT_COMPTC' , 'VL_TOTAL', 'VL_QUOTA', 'VL_PATRIM_LIQ']]
# Transformando os dtypes das colunas.
df_inf_diario['TP_FUNDO'] = df_inf_diario['TP_FUNDO'].astype(str)
df_inf_diario['CNPJ_FUNDO'] = df_inf_diario['CNPJ_FUNDO'].astype(str)
df_inf_diario['DT_COMPTC'] = pd.to_datetime(df_inf_diario['DT_COMPTC'], format='%Y-%m-%d')
# Transformando a coluna 'DT_COMPTC' em index
df_inf_diario = df_inf_diario.set_index('DT_COMPTC')

# A partir do arquivo 'inf_diario_fi_202408', a B3 mudou os nomes das colunas 'TP_FUNDO' e 'CNPJ_FUNDO' para 'TP_FUNDO_CLASSE' e 'CNPJ_FUNDO_CLASSE' e adicionou a coluna 'ID_SUBCLASSE'
paths_arquivos_filt_2 = [path for path in paths_arquivos if path[-14:-8] >= '202408']
# Lendo os arquivos parquet e juntando em apenas um df
lst_df_2 = [pd.read_parquet(path) for path in paths_arquivos_filt_2]
# Concatenando os dfs
df_inf_diario_2 = pd.concat(lst_df_2, ignore_index=True)
# Selecionando os 'Fundos de Investimentos' e 'Fundo de Investimento Financeiro'
filt_fi = df_inf_diario_2['TP_FUNDO_CLASSE'].isin(['FI', 'CLASSES - FIF'])
df_inf_diario_2 = df_inf_diario_2.loc[filt_fi]
# Selecionando as principais colunas
df_inf_diario_2 = df_inf_diario_2[['TP_FUNDO_CLASSE', 'CNPJ_FUNDO_CLASSE', 'DT_COMPTC' , 'VL_TOTAL', 'VL_QUOTA', 'VL_PATRIM_LIQ']]
# Renomeando as colunas 
df_inf_diario_2 = df_inf_diario_2.rename(columns={'TP_FUNDO_CLASSE':'TP_FUNDO', 'CNPJ_FUNDO_CLASSE':'CNPJ_FUNDO'})
# Transformando os dtypes das colunas.
df_inf_diario_2['TP_FUNDO'] = df_inf_diario_2['TP_FUNDO'].astype(str)
df_inf_diario_2['CNPJ_FUNDO'] = df_inf_diario_2['CNPJ_FUNDO'].astype(str)
df_inf_diario_2['DT_COMPTC'] = pd.to_datetime(df_inf_diario_2['DT_COMPTC'], format='%Y-%m-%d')
# Transformando a coluna 'DT_COMPTC' em index
df_inf_diario_2 = df_inf_diario_2.set_index('DT_COMPTC')

# Concatenando os dois dfs 
df_inf_diario_final = pd.concat([df_inf_diario, df_inf_diario_2])
df_inf_diario_final.tail()

,TP_FUNDO,CNPJ_FUNDO,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ
DT_COMPTC,,,,,
2025-05-26,CLASSES - FIF,97.929.213/0001-34,92852737.04,13.050745,92865019.88
2025-05-27,CLASSES - FIF,97.929.213/0001-34,93089894.83,13.083984,93101537.84
2025-05-28,CLASSES - FIF,97.929.213/0001-34,93075159.20,13.081823,93086161.23
2025-05-29,CLASSES - FIF,97.929.213/0001-34,93082234.79,13.082727,93092595.91
2025-05-30,CLASSES - FIF,97.929.213/0001-34,93107908.32,13.086245,92370190.30


**Principal Benchmark - IBOVESPA** 

In [6]:
# IBOVESPA
ibov = yf.download('^BVSP', start='2023-01-01', auto_adjust=True, multi_level_index=False)[['Open', 'Close']]

# Calculando a rentabilidade mensal do benchmark 
rentabilidade_mensal = {}
for date, group in ibov.groupby(ibov.index.to_period('M')):
    open_price_first_day = group['Open'].iloc[0]
    adj_close_last_day = group['Close'].iloc[-1]
    rentabilidade = ((adj_close_last_day / open_price_first_day) - 1) * 100
    rentabilidade_mensal[date.to_timestamp()] = round(rentabilidade, 2)

# Transformando o dicionário em um df
df_ret_ibov_mensal = pd.DataFrame(list(rentabilidade_mensal.items()), columns=['data', 'ret_ibov'])
# Transformando a coluna 'data' em index
df_ret_ibov_mensal = df_ret_ibov_mensal.set_index('data')
# Transformando o formato do index ('ano-mes') p/ conseguir concatenar com as rentabilidades dos fundos
df_ret_ibov_mensal.index = df_ret_ibov_mensal.index.to_period('M')


# Calculando a rentabilidade anual do benchmark 
rentabilidade_anual = {}
for date, group in ibov.groupby(ibov.index.to_period('Y')):
    open_price_first_day = group['Open'].iloc[0]
    adj_close_last_day = group['Close'].iloc[-1]
    rentabilidade = ((adj_close_last_day / open_price_first_day) - 1) * 100
    rentabilidade_anual[date.to_timestamp()] = round(rentabilidade, 2)

# Transformando o dicionário em um df
df_ret_ibov_anual = pd.DataFrame(list(rentabilidade_anual.items()), columns=['data', 'ret_ibov'])
# Transformando a coluna 'data' em index
df_ret_ibov_anual = df_ret_ibov_anual.set_index('data')
# Transformando o formato do index ('ano-mes') p/ conseguir concatenar com as rentabilidades dos fundos
df_ret_ibov_anual.index = df_ret_ibov_anual.index.to_period('Y')

print(df_ret_ibov_mensal)
print(df_ret_ibov_anual)

[*********************100%***********************]  1 of 1 completed

         ret_ibov
data             
2023-01      3.46
2023-02     -7.49
2023-03     -2.91
2023-04      2.50
2023-05      3.74
2023-06      9.00
2023-07      3.26
2023-08     -5.09
2023-09      0.71
2023-10     -2.93
2023-11     12.53
2023-12      5.38
2024-01     -4.79
2024-02      0.99
2024-03     -0.71
2024-04     -1.70
2024-05     -3.04
2024-06      1.48
2024-07      3.02
2024-08      6.54
2024-09     -3.08
2024-10     -1.60
2024-11     -3.12
2024-12     -4.29
2025-01      4.87
2025-02     -2.64
2025-03      6.08
2025-04      3.68
2025-05      1.45
2025-06      1.33
2025-07     -4.17
2025-08      6.40
2025-09      0.31
      ret_ibov
data          
2023     22.28
2024    -10.36
2025     17.94


## Fundos de Investimentos - Casas Famosas

### Fundo Verde

* https://maisretorno.com/fundo/verde-am-acoes-master-fia-1

In [7]:
# Rentabilidade mensal 
df_ret_mensal_verde = rentabilidade_fundo(df=df_inf_diario_final, cnpj='16.929.553/0001-63', nome_fundo='verde')[0]

# Comparação de rentabilidade entre fundo e benchmark
df_ret_mensal_verde_ibov = rentabilidade_fundo_benchmark(
    df_fundo=df_ret_mensal_verde,
    df_benchmark=df_ret_ibov_mensal,
    nome_fundo='verde',
    nome_benchmark='ibov'
)

df_ret_mensal_verde_ibov

,ret_verde,ret_ibov,performance
2023-01,0.65,3.46,-2.81
2023-02,-4.23,-7.49,3.26
2023-03,-1.75,-2.91,1.16
2023-04,-1.48,2.50,-3.98
2023-05,5.05,3.74,1.31
2023-06,7.85,9.00,-1.15
2023-07,2.09,3.26,-1.17
2023-08,-2.25,-5.09,2.84
2023-09,-0.18,0.71,-0.89
2023-10,-6.01,-2.93,-3.08


In [8]:
# Rentabilidade anual
df_ret_anual_verde = rentabilidade_fundo(df=df_inf_diario_final, cnpj='16.929.553/0001-63', nome_fundo='verde')[1]

# Comparação de rentabilidade entre fundo e benchmark
df_ret_anual_verde_ibov = rentabilidade_fundo_benchmark(
    df_fundo=df_ret_anual_verde,
    df_benchmark=df_ret_ibov_anual,
    nome_fundo='verde',
    nome_benchmark='ibov'
)

df_ret_anual_verde_ibov

,ret_verde,ret_ibov,performance
2023,11.57,22.28,-10.71
2024,-14.93,-10.36,-4.57
2025,20.79,17.94,2.85


In [9]:
verde_acoes, verde_bdr, verde_exterior, verde_cotas_fundos, verde_titulos_pub, verde_vendido_acoes = fundo_cnpj(df=df_ativos, cnpj='16.929.553/0001-63')
verde_pl = pl_fundo(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_PL_{ano}{mes}.parquet', 
    cnpj='16.929.553/0001-63'
)

# Quantos porcertos cada categoria representa do PL final
pct_acoes = round(pd.Series(verde_acoes['VL_MERC_POS_FINAL'].sum() / verde_pl.values), 4) 
pct_bdr = round(pd.Series(verde_bdr['VL_MERC_POS_FINAL'].sum() / verde_pl.values), 4) 
pct_exterior = round(pd.Series(verde_exterior['VL_MERC_POS_FINAL'].sum() / verde_pl.values), 4) 
pct_cotas_fundos = round(pd.Series(verde_cotas_fundos['VL_MERC_POS_FINAL'].sum() / verde_pl.values), 4) 
pct_titulos_pub = round(pd.Series(verde_titulos_pub['VL_MERC_POS_FINAL'].sum() / verde_pl.values), 4) 
pct_vendido_acoes = round(pd.Series(verde_vendido_acoes['VL_MERC_POS_FINAL'].sum() / verde_pl.values), 4) 

# Criando a data do portfólio
data = pd.to_datetime(f'{mes}-{ano}')

# Adicionando a informação da porcentagem de cada categoria nos dfs
# Ações
linha_pct_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_acoes}
df_linha_pct_acoes = pd.DataFrame(linha_pct_acoes)
verde_acoes = pd.concat([verde_acoes, df_linha_pct_acoes])
# Criando a coluna 'data' no df
verde_acoes['data'] = data
# Definindo a coluna 'data' como o index
verde_acoes = verde_acoes.set_index('data')

# BDRs
linha_pct_bdr = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_bdr}
df_linha_pct_bdr = pd.DataFrame(linha_pct_bdr)
verde_bdr = pd.concat([verde_bdr, df_linha_pct_bdr])
# Criando a coluna 'data' no df
verde_bdr['data'] = data
# Definindo a coluna 'data' como o index
verde_bdr = verde_bdr.set_index('data')

# Investimentos no Exterior
linha_pct_exterior = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_exterior}
df_linha_pct_exterior = pd.DataFrame(linha_pct_exterior)
verde_exterior = pd.concat([verde_exterior, df_linha_pct_exterior])
# Criando a coluna 'data' no df
verde_exterior['data'] = data
# Definindo a coluna 'data' como o index
verde_exterior = verde_exterior.set_index('data')

# Cotas de Fundos
linha_pct_cotas_fundos = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_cotas_fundos}
df_linha_pct_cotas_fundos = pd.DataFrame(linha_pct_cotas_fundos)
verde_cotas_fundos = pd.concat([verde_cotas_fundos, df_linha_pct_cotas_fundos])
# Criando a coluna 'data' no df
verde_cotas_fundos['data'] = data
# Definindo a coluna 'data' como o index
verde_cotas_fundos = verde_cotas_fundos.set_index('data')

# Títulos Públicos
linha_pct_titulos_pub = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_titulos_pub}
df_linha_pct_titulos_pub = pd.DataFrame(linha_pct_titulos_pub)
verde_titulos_pub = pd.concat([verde_titulos_pub, df_linha_pct_titulos_pub])
# Criando a coluna 'data' no df
verde_titulos_pub['data'] = data
# Definindo a coluna 'data' como o index
verde_titulos_pub = verde_titulos_pub.set_index('data')

# Vendido Ações
linha_pct_vendido_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_vendido_acoes}
df_linha_pct_vendido_acoes = pd.DataFrame(linha_pct_vendido_acoes)
verde_vendido_acoes = pd.concat([verde_vendido_acoes, df_linha_pct_vendido_acoes ])
# Criando a coluna 'data' no df
verde_vendido_acoes['data'] = data
# Definindo a coluna 'data' como o index
verde_veverde_vendido_acoes =verde_vendido_acoes.set_index('data')

In [10]:
# Lista das porcentagens de cada categoria que compõe o fundo
lst_pct_categorias_verde = [pct_acoes , pct_bdr, pct_exterior, pct_cotas_fundos, pct_titulos_pub, pct_vendido_acoes]
# Transformando em um df
df_pct_categorias_verde = pd.DataFrame(lst_pct_categorias_verde).T
# Renomeando as colunas
df_pct_categorias_verde = df_pct_categorias_verde.rename(columns={
    0:'pct_acoes',
    1:'pct_bdr',
    2:'pct_exterior',
    3:'pct_cotas_fundos',
    4:'pct_titulos_pub',
    5:'pct_vendido_acoes'
})

df_pct_categorias_verde

,pct_acoes,pct_bdr,pct_exterior,pct_cotas_fundos,pct_titulos_pub,pct_vendido_acoes
0,0.6251,0.0,0.0298,0.0,0.0,0.0


In [11]:
# Criando o arquivo excel em que cada categoria está em uma aba diferente
with pd.ExcelWriter(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//fundo_investimento//verde//fundo_verde_{ano}{mes}.xlsx') as writer:
    verde_acoes.to_excel(writer, sheet_name='acoes')
    verde_bdr.to_excel(writer, sheet_name='bdr')
    verde_exterior.to_excel(writer, sheet_name='exterior')
    verde_cotas_fundos.to_excel(writer, sheet_name='cota_fundos')
    verde_titulos_pub.to_excel(writer, sheet_name='titulos_publicos')
    verde_vendido_acoes.to_excel(writer, sheet_name='vendido_acoes')

In [12]:
# Caminho da pasta do fundo Verde
path_verde = 'C:\\Users\\vitor\\projetos_python\\python_b3\\composicao-fundos-de-investimentos\\fundo_investimento\\verde\\'

# Caminho completo de todos os arquivos que estão na pasta
paths_arquivos_verde = glob.glob(path_verde + '*.xlsx')

# Lendo cada arquivo excel
lst_dfs = []
for path in paths_arquivos_verde:
    df = pd.read_excel(path, sheet_name='acoes')
    lst_dfs.append(df)

# Concatendo os dfs de ações
portfolio_verde = pd.concat(lst_dfs, ignore_index=True)

# Retirando as ações que não tem nenhum dinheiro alocado e os NAN
portfolio_verde = portfolio_verde[portfolio_verde['VL_MERC_POS_FINAL'] != 0]
portfolio_verde = portfolio_verde.dropna()

# Definindo a coluna 'data' como o index
portfolio_verde = portfolio_verde.set_index('data')
portfolio_verde

,DENOM_SOCIAL,CD_ATIVO,PORCENTAGEM,VL_MERC_POS_FINAL
data,,,,
2023-01-01,VERDE AM LONG BIAS MASTER FUNDO DE INVESTIMENT...,SUZB3,0.129612,45251996.00
2023-01-01,VERDE AM LONG BIAS MASTER FUNDO DE INVESTIMENT...,RENT3,0.095605,33378749.35
2023-01-01,VERDE AM LONG BIAS MASTER FUNDO DE INVESTIMENT...,EQTL3,0.085939,30004152.24
2023-01-01,VERDE AM LONG BIAS MASTER FUNDO DE INVESTIMENT...,VBBR3,0.076611,26747500.00
2023-01-01,VERDE AM LONG BIAS MASTER FUNDO DE INVESTIMENT...,RAIL3,0.069380,24223002.00
...,...,...,...,...
2025-02-01,VERDE AM AÇÕES MASTER FUNDO DE INVESTIMENTO FI...,RDOR3,0.049803,7993071.49
2025-02-01,VERDE AM AÇÕES MASTER FUNDO DE INVESTIMENTO FI...,VALE3,0.044678,7170603.00
2025-02-01,VERDE AM AÇÕES MASTER FUNDO DE INVESTIMENTO FI...,PRIO3,0.028135,4515511.00


In [13]:
# Selecionando o conjunto de ações (portfólio) de cada mês - retornando um df com vários dicionários
portfolio_mensal = portfolio_verde.groupby('data')['CD_ATIVO'].apply(set)

# Convertendo o GroupBy resultante em DataFrame para facilitar a iteração
portfolio_mensal = portfolio_mensal.reset_index()

# Comparando o portfólio de cada mês
comparar_portfolios(df=portfolio_mensal, nome_fundo='Verde')

Comparando o portfólio de 01/2023 e 02/2023:
O Verde vendeu as ações: {'PETR4', 'MODL3', 'SOMA3', 'BEEF3', 'TIMS3', 'ALSO3', 'RECV3', 'CMIG3', 'YDUQ3', 'LJQQ3', 'MULT3', 'CASH3'}
O Verde comprou as ações: {'EZTC3', 'JBSS3', 'EGIE3', 'MRFG3', 'BOAS3', 'CRFB3', 'VALE3', 'PSSA3', 'LWSA3', 'EMBR3', 'CPLE6'}
--------------------------------------------------------------------------------
Comparando o portfólio de 02/2023 e 03/2023:
O Verde vendeu as ações: {'INTB3', 'GMAT3', 'EZTC3', 'MTRE3', 'LREN3', 'SQIA3', 'MRFG3', 'GOAU4', 'CIEL3', 'RDOR3', 'GGBR4', 'BBAS3', 'RAIZ4', 'CSAN3', 'EMBR3'}
O Verde comprou as ações: {'STBP3', 'BEEF3', 'ELET6', 'SMTO3', 'DIRR3', 'BRFS3'}
--------------------------------------------------------------------------------
Comparando o portfólio de 03/2023 e 04/2023:
O Verde vendeu as ações: {'ELET3', 'RRRP3', 'BRAP4', 'USIM5', 'CRFB3', 'PSSA3', 'PETZ3', 'ENJU3', 'DIRR3', 'CBAV3', 'BRPR3', 'LWSA3', 'BRKM5'}
O Verde comprou as ações: {'EZTC3', 'LREN3', 'PETR4', 'SOM

In [14]:
# Número total de ações de cada mês do portfpolio
num_total_acoes_verde = num_total_acoes(df=portfolio_verde)
num_total_acoes_verde

data
2023-01-01    71
2023-02-01    70
2023-03-01    61
2023-04-01    60
2023-05-01    59
2023-06-01    58
2023-07-01    69
2023-08-01    51
2023-09-01    46
2023-10-01    43
2023-11-01    41
2023-12-01    57
2024-01-01    59
2024-02-01    49
2024-03-01    59
2024-04-01    47
2024-05-01    31
2024-06-01    14
2024-07-01    16
2024-08-01    17
2024-09-01    17
2024-10-01    15
2024-11-01    15
2024-12-01    15
2025-01-01    15
2025-02-01    14
Name: CD_ATIVO, dtype: int64

In [15]:
# Rank das 5 maiores ações do fundo
rank_top_5_verde = rank_top_5(df=portfolio_verde)
rank_top_5_verde

data
2023-01-01    [SUZB3, RENT3, EQTL3, VBBR3, RAIL3]
2023-02-01    [SUZB3, RENT3, RAIL3, VBBR3, EQTL3]
2023-03-01    [SUZB3, RAIL3, VBBR3, ASAI3, SBSP3]
2023-04-01    [SUZB3, RAIL3, VBBR3, RENT3, EQTL3]
2023-05-01    [SUZB3, RAIL3, EQTL3, VBBR3, SBSP3]
2023-06-01    [SUZB3, RENT3, RAIL3, EQTL3, VBBR3]
2023-07-01    [SUZB3, RAIL3, RENT3, VBBR3, ITUB4]
2023-08-01    [SUZB3, RAIL3, VBBR3, SBSP3, RENT3]
2023-09-01    [SUZB3, RAIL3, SBSP3, VBBR3, RENT3]
2023-10-01    [SUZB3, RAIL3, EQTL3, ITUB4, SBSP3]
2023-11-01    [SUZB3, SBSP3, ITUB4, CPLE3, EQTL3]
2023-12-01    [SUZB3, SBSP3, EQTL3, CPLE3, ITUB4]
2024-01-01    [RAIL3, SUZB3, SBSP3, ITUB4, EQTL3]
2024-02-01    [SUZB3, RAIL3, SBSP3, ITUB4, EQTL3]
2024-03-01    [SUZB3, SBSP3, ITUB4, ELET3, ENEV3]
2024-04-01    [SUZB3, PETR4, ITUB4, SBSP3, ENEV3]
2024-05-01    [ITUB4, SUZB3, ELET3, SBSP3, PRIO3]
2024-06-01    [ITUB4, SUZB3, ELET3, PRIO3, SBSP3]
2024-07-01    [ITUB4, SBSP3, ELET3, PRIO3, SUZB3]
2024-08-01    [ELET3, SBSP3, SUZB3, ITUB4, PR

In [ ]:
# Plotando o portfólio de cada mês
plot_portfolio(df=portfolio_verde, nome_fundo='Verde')

### Fundo Dynamo

* https://maisretorno.com/fundo/dynamo-cougar-master-fia

In [16]:
# Rentabilidade mensal
df_ret_mensal_dynamo = rentabilidade_fundo(df=df_inf_diario_final, cnpj='37.916.879/0001-26', nome_fundo='dynamo')[0]

# Comparação de rentabilidade entre fundo e benchmark
df_ret_mensal_dynamo_ibov = rentabilidade_fundo_benchmark(
    df_fundo=df_ret_mensal_dynamo,
    df_benchmark=df_ret_ibov_mensal,
    nome_fundo='dynamo',
    nome_benchmark='ibov'
)

df_ret_mensal_dynamo_ibov

,ret_dynamo,ret_ibov,performance
2023-01,4.80,3.46,1.34
2023-02,-4.66,-7.49,2.83
2023-03,-6.02,-2.91,-3.11
2023-04,-0.24,2.50,-2.74
2023-05,12.20,3.74,8.46
2023-06,8.40,9.00,-0.60
2023-07,5.00,3.26,1.74
2023-08,-6.35,-5.09,-1.26
2023-09,-0.77,0.71,-1.48
2023-10,-4.90,-2.93,-1.97


In [17]:
# Rentabilidade anual
df_ret_anual_dynamo = rentabilidade_fundo(df=df_inf_diario_final, cnpj='37.916.879/0001-26', nome_fundo='dynamo')[1]

# Comparação de rentabilidade entre fundo e benchmark
df_ret_anual_dynamo_ibov = rentabilidade_fundo_benchmark(
    df_fundo=df_ret_anual_dynamo,
    df_benchmark=df_ret_ibov_anual,
    nome_fundo='dynamo',
    nome_benchmark='ibov'
)

df_ret_anual_dynamo_ibov

,ret_dynamo,ret_ibov,performance
2023,24.92,22.28,2.64
2024,-9.84,-10.36,0.52
2025,21.95,17.94,4.01


In [18]:
dynamo_acoes, dynamo_bdr, dynamo_exterior, dynamo_cotas_fundos, dynamo_titulos_pub, dynamo_vendido_acoes = fundo_cnpj(df=df_ativos, cnpj='37.916.879/0001-26')
dynamo_pl = pl_fundo(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_PL_{ano}{mes}.parquet',
    cnpj='37.916.879/0001-26'
)

# Quantos porcertos cada categoria representa do PL final
pct_acoes = round(pd.Series(dynamo_acoes['VL_MERC_POS_FINAL'].sum() / dynamo_pl.values), 4) 
pct_bdr = round(pd.Series(dynamo_bdr['VL_MERC_POS_FINAL'].sum() / dynamo_pl.values), 4) 
pct_exterior = round(pd.Series(dynamo_exterior['VL_MERC_POS_FINAL'].sum() / dynamo_pl.values), 4) 
pct_cotas_fundos = round(pd.Series(dynamo_cotas_fundos['VL_MERC_POS_FINAL'].sum() / dynamo_pl.values), 4) 
pct_titulos_pub = round(pd.Series(dynamo_titulos_pub['VL_MERC_POS_FINAL'].sum() / dynamo_pl.values), 4) 
pct_vendido_acoes = round(pd.Series(dynamo_vendido_acoes['VL_MERC_POS_FINAL'].sum() / dynamo_pl.values), 4) 

# Criando a data do portfólio
data = pd.to_datetime(f'{mes}-{ano}')

# Adicionando a informação da porcentagem de cada categoria nos dfs
# Ações
linha_pct_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_acoes}
df_linha_pct_acoes = pd.DataFrame(linha_pct_acoes)
dynamo_acoes = pd.concat([dynamo_acoes, df_linha_pct_acoes])
# Criando a coluna 'data' no df
dynamo_acoes['data'] = data
# Definindo a coluna 'data' como o index
dynamo_acoes = dynamo_acoes.set_index('data')

# BDRs
linha_pct_bdr = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_bdr}
df_linha_pct_bdr = pd.DataFrame(linha_pct_bdr)
dynamo_bdr = pd.concat([dynamo_bdr, df_linha_pct_bdr])
# Criando a coluna 'data' no df
dynamo_bdr['data'] = data
# Definindo a coluna 'data' como o index
dynamo_bdr = dynamo_bdr.set_index('data')

# Investimentos no Exterior
linha_pct_exterior = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_exterior}
df_linha_pct_exterior = pd.DataFrame(linha_pct_exterior)
dynamo_exterior = pd.concat([dynamo_exterior, df_linha_pct_exterior])
# Criando a coluna 'data' no df
dynamo_exterior['data'] = data
# Definindo a coluna 'data' como o index
dynamo_exterior = dynamo_exterior.set_index('data')

# Cotas de Fundos
linha_pct_cotas_fundos = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_cotas_fundos}
df_linha_pct_cotas_fundos = pd.DataFrame(linha_pct_cotas_fundos)
dynamo_cotas_fundos = pd.concat([dynamo_cotas_fundos, df_linha_pct_cotas_fundos])
# Criando a coluna 'data' no df
dynamo_cotas_fundos['data'] = data
# Definindo a coluna 'data' como o index
dynamo_codynamo_cotas_fundos =dynamo_cotas_fundos.set_index('data')

# Títulos Públicos
linha_pct_titulos_pub = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_titulos_pub}
df_linha_pct_titulos_pub = pd.DataFrame(linha_pct_titulos_pub)
dynamo_titulos_pub = pd.concat([dynamo_titulos_pub, df_linha_pct_titulos_pub])
# Criando a coluna 'data' no df
dynamo_titulos_pub['data'] = data
# Definindo a coluna 'data' como o index
dynamo_titulos_pub = dynamo_titulos_pub.set_index('data')

# Vendido Ações
linha_pct_vendido_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_vendido_acoes}
df_linha_pct_vendido_acoes = pd.DataFrame(linha_pct_vendido_acoes)
dynamo_vendido_acoes = pd.concat([dynamo_vendido_acoes, df_linha_pct_vendido_acoes])
# Criando a coluna 'data' no df
dynamo_vendido_acoes['data'] = data
# Definindo a coluna 'data' como o index
dynamo_vendido_acoes = dynamo_vendido_acoes.set_index('data')

In [19]:
# Lista das porcentagens de cada categoria que compõe o fundo
lst_pct_categorias_dynamo = [pct_acoes , pct_bdr, pct_exterior, pct_cotas_fundos, pct_titulos_pub, pct_vendido_acoes]
# Transformando em um df
df_pct_categorias_dynamo = pd.DataFrame(lst_pct_categorias_dynamo).T
# Renomeando as colunas
df_pct_categorias_dynamo = df_pct_categorias_dynamo.rename(columns={
    0:'pct_acoes',
    1:'pct_bdr',
    2:'pct_exterior',
    3:'pct_cotas_fundos',
    4:'pct_titulos_pub',
    5:'pct_vendido_acoes'
})

df_pct_categorias_dynamo

,pct_acoes,pct_bdr,pct_exterior,pct_cotas_fundos,pct_titulos_pub,pct_vendido_acoes
0,0.8742,0.001,0.1543,0.0,0.0,0.0892


In [20]:
# Criando o arquivo excel em que cada categoria está em uma aba diferente
with pd.ExcelWriter(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//fundo_investimento//dynamo//fundo_dynamo_{ano}{mes}.xlsx') as writer:
    dynamo_acoes.to_excel(writer, sheet_name='acoes')
    dynamo_bdr.to_excel(writer, sheet_name='bdr')
    dynamo_exterior.to_excel(writer, sheet_name='exterior')
    dynamo_cotas_fundos.to_excel(writer, sheet_name='cota_fundos')
    dynamo_titulos_pub.to_excel(writer, sheet_name='titulos_publicos')
    dynamo_vendido_acoes.to_excel(writer, sheet_name='vendido_acoes')

In [21]:
# Caminho da pasta do fundo Dynamo
path_dynamo = 'C:\\Users\\vitor\\projetos_python\\python_b3\\composicao-fundos-de-investimentos\\fundo_investimento\\dynamo\\'

# Caminho completo de todos os arquivos que estão na pasta
paths_arquivos_dynamo = glob.glob(path_dynamo + '*.xlsx')

lst_dfs = []
for path in paths_arquivos_dynamo:
    df = pd.read_excel(path, sheet_name='acoes')
    lst_dfs.append(df)

# Concatendo os dfs de ações
portfolio_dynamo = pd.concat(lst_dfs, ignore_index=True)

# Retirando as ações que não tem nenhum dinheiro alocado e os NAN
portfolio_dynamo = portfolio_dynamo[portfolio_dynamo['VL_MERC_POS_FINAL'] != 0]
portfolio_dynamo = portfolio_dynamo.dropna()

# Definindo a coluna 'data' como o index
portfolio_dynamo = portfolio_dynamo.set_index('data')
portfolio_dynamo

,DENOM_SOCIAL,CD_ATIVO,PORCENTAGEM,VL_MERC_POS_FINAL
data,,,,
2023-01-01,DYNAMO COUGAR MASTER FUNDO DE INVESTIMENTO EM ...,ENEV3,0.147659,9.885924e+08
2023-01-01,DYNAMO COUGAR MASTER FUNDO DE INVESTIMENTO EM ...,VBBR3,0.110783,7.417016e+08
2023-01-01,DYNAMO COUGAR MASTER FUNDO DE INVESTIMENTO EM ...,CSAN3,0.106247,7.113367e+08
2023-01-01,DYNAMO COUGAR MASTER FUNDO DE INVESTIMENTO EM ...,RENT3,0.100129,6.703738e+08
2023-01-01,DYNAMO COUGAR MASTER FUNDO DE INVESTIMENTO EM ...,NTCO3,0.093790,6.279359e+08
...,...,...,...,...
2025-02-01,DYNAMO COUGAR MASTER FUNDO DE INVESTIMENTO FIN...,AURE3,0.001315,9.177820e+06
2025-02-01,DYNAMO COUGAR MASTER FUNDO DE INVESTIMENTO FIN...,GGBR4,0.001302,9.091212e+06
2025-02-01,DYNAMO COUGAR MASTER FUNDO DE INVESTIMENTO FIN...,USIM3,0.000658,4.589558e+06


In [22]:
# Selecionando o conjunto de ações (portfólio) de cada mês - retornando um df com vários dicionários
portfolio_mensal = portfolio_dynamo.groupby('data')['CD_ATIVO'].apply(set)

# Convertendo o GroupBy resultante em DataFrame para facilitar a iteração
portfolio_mensal = portfolio_mensal.reset_index()

# Comparando o portfólio de cada mês
comparar_portfolios(df=portfolio_mensal, nome_fundo='Dynamo')

Comparando o portfólio de 01/2023 e 02/2023:
O Dynamo vendeu as ações: {'PETR4', 'PETR3', 'RENT3', 'ITUB4'}
O Dynamo comprou as ações: {'CCRO3'}
--------------------------------------------------------------------------------
Comparando o portfólio de 02/2023 e 03/2023:
O Dynamo vendeu as ações: {'GGBR4', 'ENJU3'}
O Dynamo comprou as ações: {'ASAI3', 'OPCT3'}
--------------------------------------------------------------------------------
Comparando o portfólio de 03/2023 e 04/2023:
O Dynamo vendeu as ações: set()
O Dynamo comprou as ações: {'GGBR4'}
--------------------------------------------------------------------------------
Comparando o portfólio de 04/2023 e 05/2023:
O Dynamo vendeu as ações: set()
O Dynamo comprou as ações: {'MOVI3', 'RENT3'}
--------------------------------------------------------------------------------
Comparando o portfólio de 05/2023 e 06/2023:
O Dynamo vendeu as ações: {'B3SA3'}
O Dynamo comprou as ações: {'UGPA3', 'RAIZ4'}
-------------------------------

In [23]:
# Número total de ações de cada mês do portfpolio
num_total_acoes_dynamo = num_total_acoes(df=portfolio_dynamo)
num_total_acoes_dynamo

data
2023-01-01    30
2023-02-01    27
2023-03-01    27
2023-04-01    28
2023-05-01    30
2023-06-01    31
2023-07-01    31
2023-08-01    31
2023-09-01    31
2023-10-01    31
2023-11-01    31
2023-12-01    28
2024-01-01    29
2024-02-01    27
2024-03-01    24
2024-04-01    26
2024-05-01    26
2024-06-01    27
2024-07-01    27
2024-08-01    27
2024-09-01    24
2024-10-01    23
2024-11-01    25
2024-12-01    27
2025-01-01    30
2025-02-01    28
Name: CD_ATIVO, dtype: int64

In [24]:
# Rank das 5 maiores ações do fundo
rank_top_5_dynamo = rank_top_5(df=portfolio_dynamo)
rank_top_5_dynamo

data
2023-01-01    [ENEV3, VBBR3, CSAN3, RENT3, NTCO3]
2023-02-01    [ENEV3, VBBR3, CSAN3, NTCO3, RDOR3]
2023-03-01    [ENEV3, VBBR3, CSAN3, NTCO3, RDOR3]
2023-04-01    [ENEV3, CSAN3, VBBR3, RDOR3, NTCO3]
2023-05-01    [ENEV3, VBBR3, RDOR3, SUZB3, NTCO3]
2023-06-01    [ENEV3, VBBR3, RDOR3, NTCO3, CSAN3]
2023-07-01    [ENEV3, RDOR3, VBBR3, NTCO3, CSAN3]
2023-08-01    [ENEV3, VBBR3, RDOR3, NTCO3, CSAN3]
2023-09-01    [ENEV3, VBBR3, RDOR3, NTCO3, CSAN3]
2023-10-01    [VBBR3, ENEV3, RDOR3, NTCO3, CSAN3]
2023-11-01    [VBBR3, ENEV3, RDOR3, NTCO3, CSAN3]
2023-12-01    [VBBR3, ENEV3, RDOR3, NTCO3, SUZB3]
2024-01-01    [VBBR3, ENEV3, RDOR3, NTCO3, SUZB3]
2024-02-01    [VBBR3, ENEV3, NTCO3, RDOR3, SUZB3]
2024-03-01    [VBBR3, ENEV3, NTCO3, SUZB3, RDOR3]
2024-04-01    [VBBR3, ENEV3, NTCO3, RDOR3, SUZB3]
2024-05-01    [VBBR3, ENEV3, RDOR3, NTCO3, SUZB3]
2024-06-01    [ENEV3, VBBR3, NTCO3, RDOR3, SUZB3]
2024-07-01    [VBBR3, ENEV3, RDOR3, SUZB3, NTCO3]
2024-08-01    [VBBR3, ENEV3, SUZB3, RDOR3, NT

In [ ]:
# Plotando o portfólio de cada mês
plot_portfolio(df=portfolio_dynamo, nome_fundo='Dynamo')

### Fundo IP

* https://maisretorno.com/fundo/ip-participacoes-master-fia-bdr-nivel-i

In [25]:
# Rentabilidade mensal
df_ret_mensal_ip = rentabilidade_fundo(df=df_inf_diario_final, cnpj='11.435.298/0001-89', nome_fundo='ip')[0]

# Comparação de rentabilidade entre fundo e benchmark
df_ret_mensal_ip_ibov = rentabilidade_fundo_benchmark(
    df_fundo=df_ret_mensal_ip,
    df_benchmark=df_ret_ibov_mensal,
    nome_fundo='ip',
    nome_benchmark='ibov'
)

df_ret_mensal_ip_ibov

,ret_ip,ret_ibov,performance
2023-01,5.11,3.46,1.65
2023-02,-5.15,-7.49,2.34
2023-03,0.56,-2.91,3.47
2023-04,3.51,2.50,1.01
2023-05,3.03,3.74,-0.71
2023-06,7.54,9.00,-1.46
2023-07,4.21,3.26,0.95
2023-08,-0.94,-5.09,4.15
2023-09,-1.87,0.71,-2.58
2023-10,-3.69,-2.93,-0.76


In [26]:
# Rentabilidade anual
df_ret_anual_ip = rentabilidade_fundo(df=df_inf_diario_final, cnpj='11.435.298/0001-89', nome_fundo='ip')[1]

# Comparação de rentabilidade entre fundo e benchmark
df_ret_anual_ip_ibov = rentabilidade_fundo_benchmark(
    df_fundo=df_ret_anual_ip,
    df_benchmark=df_ret_ibov_anual,
    nome_fundo='ip',
    nome_benchmark='ibov'
)

df_ret_anual_ip_ibov

,ret_ip,ret_ibov,performance
2023,30.94,22.28,8.66
2024,7.27,-10.36,17.63
2025,19.41,17.94,1.47


In [27]:
ip_acoes, ip_bdr, ip_exterior, ip_cotas_fundos, ip_titulos_pub, ip_vendido_acoes = fundo_cnpj(df=df_ativos, cnpj='11.435.298/0001-89')
ip_pl = pl_fundo(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_PL_{ano}{mes}.parquet', 
    cnpj='11.435.298/0001-89'
)

# Quantos porcertos cada categoria representa do PL final
pct_acoes = round(pd.Series(ip_acoes['VL_MERC_POS_FINAL'].sum() / ip_pl.values), 4) 
pct_bdr = round(pd.Series(ip_bdr['VL_MERC_POS_FINAL'].sum() / ip_pl.values), 4) 
pct_exterior = round(pd.Series(ip_exterior['VL_MERC_POS_FINAL'].sum() / ip_pl.values), 4) 
pct_cotas_fundos = round(pd.Series(ip_cotas_fundos['VL_MERC_POS_FINAL'].sum() / ip_pl.values), 4) 
pct_titulos_pub = round(pd.Series(ip_titulos_pub['VL_MERC_POS_FINAL'].sum() / ip_pl.values), 4) 
pct_vendido_acoes = round(pd.Series(ip_vendido_acoes['VL_MERC_POS_FINAL'].sum() / ip_pl.values), 4) 

# Criando a data do portfólio
data = pd.to_datetime(f'{mes}-{ano}')

# Adicionando a informação da porcentagem de cada categoria nos dfs
# Ações
linha_pct_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_acoes}
df_linha_pct_acoes = pd.DataFrame(linha_pct_acoes)
ip_acoes = pd.concat([ip_acoes, df_linha_pct_acoes])
# Criando a coluna 'data' no df
ip_acoes['data'] = data
# Definindo a coluna 'data' como o index
ip_acoes = ip_acoes.set_index('data')

# BDRs
linha_pct_bdr = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_bdr}
df_linha_pct_bdr = pd.DataFrame(linha_pct_bdr)
ip_bdr = pd.concat([ip_bdr, df_linha_pct_bdr])
# Criando a coluna 'data' no df
ip_bdr['data'] = data
# Definindo a coluna 'data' como o index
ip_bdr = ip_bdr.set_index('data')

# Investimentos no Exterior
linha_pct_exterior = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_exterior}
df_linha_pct_exterior = pd.DataFrame(linha_pct_exterior)
ip_exterior = pd.concat([ip_exterior, df_linha_pct_exterior])
# Criando a coluna 'data' no df
ip_exterior['data'] = data
# Definindo a coluna 'data' como o index
ip_exterior = ip_exterior.set_index('data')

# Cotas de Fundos
linha_pct_cotas_fundos = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_cotas_fundos}
df_linha_pct_cotas_fundos = pd.DataFrame(linha_pct_cotas_fundos)
ip_cotas_fundos = pd.concat([ip_cotas_fundos, df_linha_pct_cotas_fundos])
# Criando a coluna 'data' no df
ip_cotas_fundos['data'] = data
# Definindo a coluna 'data' como o index
ip_cotas_fundos = ip_cotas_fundos.set_index('data')

# Títulos Públicos
linha_pct_titulos_pub = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_titulos_pub}
df_linha_pct_titulos_pub = pd.DataFrame(linha_pct_titulos_pub)
ip_titulos_pub = pd.concat([ip_titulos_pub, df_linha_pct_titulos_pub])
# Criando a coluna 'data' no df
ip_titulos_pub['data'] = data
# Definindo a coluna 'data' como o index
ip_titulos_pub = ip_titulos_pub.set_index('data')

# Vendido Ações
linha_pct_vendido_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_vendido_acoes}
df_linha_pct_vendido_acoes = pd.DataFrame(linha_pct_vendido_acoes)
ip_vendido_acoes = pd.concat([ip_vendido_acoes, df_linha_pct_vendido_acoes])
# Criando a coluna 'data' no df
ip_vendido_acoes['data'] = data
# Definindo a coluna 'data' como o index
ip_vendido_acoes = ip_vendido_acoes.set_index('data')

In [28]:
# Lista das porcentagens de cada categoria que compõe o fundo
lst_pct_categorias_ip = [pct_acoes , pct_bdr, pct_exterior, pct_cotas_fundos, pct_titulos_pub, pct_vendido_acoes]
# Transformando em um df
df_pct_categorias_ip = pd.DataFrame(lst_pct_categorias_ip).T
# Renomeando as colunas
df_pct_categorias_ip = df_pct_categorias_ip.rename(columns={
    0:'pct_acoes',
    1:'pct_bdr',
    2:'pct_exterior',
    3:'pct_cotas_fundos',
    4:'pct_titulos_pub',
    5:'pct_vendido_acoes'
})

df_pct_categorias_ip

,pct_acoes,pct_bdr,pct_exterior,pct_cotas_fundos,pct_titulos_pub,pct_vendido_acoes
0,0.2641,0.1259,0.0,0.0,0.0,0.0


In [29]:
# Criando o arquivo excel em que cada categoria está em uma aba diferente
with pd.ExcelWriter(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//fundo_investimento//ip//fundo_ip_{ano}{mes}.xlsx') as writer:
    ip_acoes.to_excel(writer, sheet_name='acoes')
    ip_bdr.to_excel(writer, sheet_name='bdr')
    ip_exterior.to_excel(writer, sheet_name='exterior')
    ip_cotas_fundos.to_excel(writer, sheet_name='cota_fundos')
    ip_titulos_pub.to_excel(writer, sheet_name='titulos_publicos')
    ip_vendido_acoes.to_excel(writer, sheet_name='vendido_acoes')

In [30]:
# Caminho da pasta do fundo IP
path_ip = 'C:\\Users\\vitor\\projetos_python\\python_b3\\composicao-fundos-de-investimentos\\fundo_investimento\\ip\\'

# Caminho completo de todos os arquivos que estão na pasta
paths_arquivos_ip = glob.glob(path_ip + '*.xlsx')

lst_dfs = []
for path in paths_arquivos_ip:
    df = pd.read_excel(path, sheet_name='acoes')
    lst_dfs.append(df)

# Concatendo os dfs de acoes
portfolio_ip = pd.concat(lst_dfs, ignore_index=True)

# Retirando as acoes que não tem nenhum dinheiro alocado e os NAN
portfolio_ip = portfolio_ip[portfolio_ip['VL_MERC_POS_FINAL'] != 0]
portfolio_ip = portfolio_ip.dropna()

# Definindo a coluna 'data' como o index
portfolio_ip = portfolio_ip.set_index('data')
portfolio_ip

,DENOM_SOCIAL,CD_ATIVO,PORCENTAGEM,VL_MERC_POS_FINAL
data,,,,
2023-01-01,IP PARTICIPAÇÕES MASTER FUNDO DE INVESTIMENTO ...,ITSA4,0.360088,30610186.49
2023-01-01,IP PARTICIPAÇÕES MASTER FUNDO DE INVESTIMENTO ...,EQTL3,0.351429,29874167.46
2023-01-01,IP PARTICIPAÇÕES MASTER FUNDO DE INVESTIMENTO ...,HAPV3,0.207363,17627430.30
2023-01-01,IP PARTICIPAÇÕES MASTER FUNDO DE INVESTIMENTO ...,MULT3,0.044781,3806756.85
2023-01-01,IP PARTICIPAÇÕES MASTER FUNDO DE INVESTIMENTO ...,RDOR3,0.036338,3089036.50
...,...,...,...,...
2025-02-01,IP PARTICIPAÇÕES MASTER FUNDO DE INVESTIMENTO ...,EQTL3,0.190507,29685572.64
2025-02-01,IP PARTICIPAÇÕES MASTER FUNDO DE INVESTIMENTO ...,RENT3,0.174513,27193297.30
2025-02-01,IP PARTICIPAÇÕES MASTER FUNDO DE INVESTIMENTO ...,MULT3,0.150938,23519768.90


In [31]:
# Selecionando o conjunto de ações (portfólio) de cada mês - retornando um df com vários dicionários
portfolio_mensal = portfolio_ip.groupby('data')['CD_ATIVO'].apply(set)

# Convertendo o GroupBy resultante em DataFrame para facilitar a iteração
portfolio_mensal = portfolio_mensal.reset_index()

# Comparando o portfólio de cada mês
comparar_portfolios(df=portfolio_mensal, nome_fundo='IP')

Comparando o portfólio de 01/2023 e 02/2023:
O IP vendeu as ações: {'MULT3'}
O IP comprou as ações: set()
--------------------------------------------------------------------------------
Comparando o portfólio de 02/2023 e 03/2023:
O IP vendeu as ações: {'HAPV3', 'RDOR3'}
O IP comprou as ações: {'MULT3', 'ITUB4'}
--------------------------------------------------------------------------------
Comparando o portfólio de 03/2023 e 04/2023:
O IP vendeu as ações: set()
O IP comprou as ações: set()
--------------------------------------------------------------------------------
Comparando o portfólio de 04/2023 e 05/2023:
O IP vendeu as ações: set()
O IP comprou as ações: {'VIVA3'}
--------------------------------------------------------------------------------
Comparando o portfólio de 05/2023 e 06/2023:
O IP vendeu as ações: set()
O IP comprou as ações: {'ELET3', 'RENT3'}
--------------------------------------------------------------------------------
Comparando o portfólio de 06/2023 e 07

In [32]:
# Número total de ações de cada mês do portfpolio
num_total_acoes_ip = num_total_acoes(df=portfolio_ip)
num_total_acoes_ip

data
2023-01-01    5
2023-02-01    4
2023-03-01    4
2023-04-01    4
2023-05-01    5
2023-06-01    7
2023-07-01    7
2023-08-01    7
2023-09-01    7
2023-10-01    7
2023-11-01    7
2023-12-01    7
2024-01-01    8
2024-02-01    7
2024-03-01    7
2024-04-01    7
2024-05-01    8
2024-06-01    8
2024-07-01    8
2024-08-01    7
2024-09-01    6
2024-10-01    6
2024-11-01    6
2024-12-01    6
2025-01-01    6
2025-02-01    6
Name: CD_ATIVO, dtype: int64

In [33]:
# Rank das 5 maiores ações do fundo
rank_top_5_ip = rank_top_5(df=portfolio_ip)
rank_top_5_ip

data
2023-01-01    [ITSA4, EQTL3, HAPV3, MULT3, RDOR3]
2023-02-01           [EQTL3, ITSA4, RDOR3, HAPV3]
2023-03-01           [EQTL3, ITSA4, ITUB4, MULT3]
2023-04-01           [EQTL3, ITSA4, ITUB4, MULT3]
2023-05-01    [EQTL3, ITSA4, ITUB4, VIVA3, MULT3]
2023-06-01    [EQTL3, ITSA4, RENT3, VIVA3, ELET3]
2023-07-01    [ITSA4, EQTL3, VIVA3, ITUB4, RENT3]
2023-08-01    [ITSA4, EQTL3, ITUB4, RENT3, VIVA3]
2023-09-01    [ITSA4, RENT3, EQTL3, ELET3, VIVA3]
2023-10-01    [RENT3, ITSA4, EQTL3, ELET3, ITUB4]
2023-11-01    [RENT3, ITSA4, EQTL3, ITUB4, VIVA3]
2023-12-01    [RENT3, ITSA4, EQTL3, VIVA3, ITUB4]
2024-01-01    [RENT3, ITSA4, EQTL3, ITUB4, MULT3]
2024-02-01    [RENT3, ITSA4, EQTL3, ITUB4, VIVA3]
2024-03-01    [RENT3, EQTL3, ITSA4, ITUB4, VIVA3]
2024-04-01    [RENT3, EQTL3, ITUB4, ITSA4, VIVA3]
2024-05-01    [ITSA4, RENT3, EQTL3, VIVA3, GGPS3]
2024-06-01    [ITSA4, RENT3, EQTL3, GGPS3, VIVA3]
2024-07-01    [RENT3, ITSA4, GGPS3, VIVA3, EQTL3]
2024-08-01    [ITSA4, EQTL3, RENT3, GGPS3, MU

In [ ]:
# Plotando o portfólio de cada mês
plot_portfolio(df=portfolio_ip, nome_fundo='IP')

### Fundo Squadra

* https://maisretorno.com/fundo/squadra-master-long-biased-fia
* Famosos pelos shorts de:
    * IRBR - 11/2019
    * AERI3 - 01/2021
    * Nubank

In [34]:
# Rentabilidade mensal
df_ret_mensal_squadra = rentabilidade_fundo(df=df_inf_diario_final, cnpj='09.412.648/0001-40', nome_fundo='squadra')[0]

# Comparação de rentabilidade entre fundo e benchmark
df_ret_mensal_squadra_ibov = rentabilidade_fundo_benchmark(
    df_fundo=df_ret_mensal_squadra,
    df_benchmark=df_ret_ibov_mensal,
    nome_fundo='squadra',
    nome_benchmark='ibov'
)

df_ret_mensal_squadra_ibov

,ret_squadra,ret_ibov,performance
2023-01,5.23,3.46,1.77
2023-02,-6.62,-7.49,0.87
2023-03,-0.20,-2.91,2.71
2023-04,3.92,2.50,1.42
2023-05,8.83,3.74,5.09
2023-06,8.13,9.00,-0.87
2023-07,4.00,3.26,0.74
2023-08,2.24,-5.09,7.33
2023-09,1.13,0.71,0.42
2023-10,-0.69,-2.93,2.24


In [35]:
# Rentabilidade anual
df_ret_anual_squadra = rentabilidade_fundo(df=df_inf_diario_final, cnpj='09.412.648/0001-40', nome_fundo='squadra')[1]

# Comparação de rentabilidade entre fundo e benchmark
df_ret_anual_squadra_ibov = rentabilidade_fundo_benchmark(
    df_fundo=df_ret_anual_squadra,
    df_benchmark=df_ret_ibov_anual,
    nome_fundo='squadra',
    nome_benchmark='ibov'
)

df_ret_anual_squadra_ibov

,ret_squadra,ret_ibov,performance
2023,44.86,22.28,22.58
2024,-19.27,-10.36,-8.91
2025,29.60,17.94,11.66


In [36]:
squadra_acoes, squadra_bdr, squadra_exterior, squadra_cotas_fundos, squadra_titulos_pub, squadra_vendido_acoes = fundo_cnpj(df=df_ativos, cnpj='09.412.648/0001-40')
squadra_pl = pl_fundo(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_PL_{ano}{mes}.parquet', 
    cnpj='09.412.648/0001-40'
)

# Quantos porcertos cada categoria representa do PL final
pct_acoes = round(pd.Series(squadra_acoes['VL_MERC_POS_FINAL'].sum() / squadra_pl.values), 4) 
pct_bdr = round(pd.Series(squadra_bdr['VL_MERC_POS_FINAL'].sum() / squadra_pl.values), 4) 
pct_exterior = round(pd.Series(squadra_exterior['VL_MERC_POS_FINAL'].sum() / squadra_pl.values), 4) 
pct_cotas_fundos = round(pd.Series(squadra_cotas_fundos['VL_MERC_POS_FINAL'].sum() / squadra_pl.values), 4) 
pct_titulos_pub = round(pd.Series(squadra_titulos_pub['VL_MERC_POS_FINAL'].sum() / squadra_pl.values), 4) 
pct_vendido_acoes = round(pd.Series(squadra_vendido_acoes['VL_MERC_POS_FINAL'].sum() / squadra_pl.values), 4) 

# Criando a data do portfólio
data = pd.to_datetime(f'{mes}-{ano}')

# Adicionando a informação da porcentagem de cada categoria nos dfs
# Ações
linha_pct_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_acoes}
df_linha_pct_acoes = pd.DataFrame(linha_pct_acoes)
squadra_acoes = pd.concat([squadra_acoes, df_linha_pct_acoes], ignore_index=True)
# Criando a coluna 'data' no df
squadra_acoes['data'] = data
# Definindo a coluna 'data' como o index
squadra_acoes = squadra_acoes.set_index('data')

# BDRs
linha_pct_bdr = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_bdr}
df_linha_pct_bdr = pd.DataFrame(linha_pct_bdr)
squadra_bdr = pd.concat([squadra_bdr, df_linha_pct_bdr], ignore_index=True)
# Criando a coluna 'data' no df
squadra_bdr['data'] = data
# Definindo a coluna 'data' como o index
squadra_bdr = squadra_bdr.set_index('data')

# Investimentos no Exterior
linha_pct_exterior = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_exterior}
df_linha_pct_exterior = pd.DataFrame(linha_pct_exterior)
squadra_exterior = pd.concat([squadra_exterior, df_linha_pct_exterior], ignore_index=True)
# Criando a coluna 'data' no df
squadra_exterior['data'] = data
# Definindo a coluna 'data' como o index
squadra_exterior = squadra_exterior.set_index('data')

# Cotas de Fundos
linha_pct_cotas_fundos = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_cotas_fundos}
df_linha_pct_cotas_fundos = pd.DataFrame(linha_pct_cotas_fundos)
squadra_cotas_fundos = pd.concat([squadra_cotas_fundos, df_linha_pct_cotas_fundos], ignore_index=True)
# Criando a coluna 'data' no df
squadra_cotas_fundos['data'] = data
# Definindo a coluna 'data' como o index
squadra_cotas_fundos = squadra_cotas_fundos.set_index('data')

# Títulos Públicos
linha_pct_titulos_pub = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_titulos_pub}
df_linha_pct_titulos_pub = pd.DataFrame(linha_pct_titulos_pub)
squadra_titulos_pub = pd.concat([squadra_titulos_pub, df_linha_pct_titulos_pub], ignore_index=True)
# Criando a coluna 'data' no df
squadra_titulos_pub['data'] = data
# Definindo a coluna 'data' como o index
squadra_titulos_pub = squadra_titulos_pub.set_index('data')

# Vendido Ações
linha_pct_vendido_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_vendido_acoes}
df_linha_pct_vendido_acoes = pd.DataFrame(linha_pct_vendido_acoes)
squadra_vendido_acoes = pd.concat([squadra_vendido_acoes, df_linha_pct_vendido_acoes], ignore_index=True)
# Criando a coluna 'data' no df
squadra_vendido_acoes['data'] = data
# Definindo a coluna 'data' como o index
squadra_vendido_acoes = squadra_vendido_acoes.set_index('data')

In [37]:
# Lista das porcentagens de cada categoria que compõe o fundo
lst_pct_categorias_squadra = [pct_acoes , pct_bdr, pct_exterior, pct_cotas_fundos, pct_titulos_pub, pct_vendido_acoes]
# Transformando em um df
df_pct_categorias_squadra = pd.DataFrame(lst_pct_categorias_squadra).T
# Renomeando as colunas
df_pct_categorias_squadra = df_pct_categorias_squadra.rename(columns={
    0:'pct_acoes',
    1:'pct_bdr',
    2:'pct_exterior',
    3:'pct_cotas_fundos',
    4:'pct_titulos_pub',
    5:'pct_vendido_acoes'
})

df_pct_categorias_squadra

,pct_acoes,pct_bdr,pct_exterior,pct_cotas_fundos,pct_titulos_pub,pct_vendido_acoes
0,0.6299,0.0664,0.1042,0.0,0.0,0.0242


In [38]:
# Criando o arquivo excel em que cada categoria está em uma aba diferente
with pd.ExcelWriter(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//fundo_investimento//squadra//fundo_squadra_{ano}{mes}.xlsx') as writer:
    squadra_acoes.to_excel(writer, sheet_name='acoes')
    squadra_bdr.to_excel(writer, sheet_name='bdr')
    squadra_exterior.to_excel(writer, sheet_name='exterior')
    squadra_cotas_fundos.to_excel(writer, sheet_name='cota_fundos')
    squadra_titulos_pub.to_excel(writer, sheet_name='titulos_publicos')
    squadra_vendido_acoes.to_excel(writer, sheet_name='vendido_acoes')

In [39]:
# Caminho da pasta do fundo Squadra
path_squadra = 'C:\\Users\\vitor\\projetos_python\\python_b3\\composicao-fundos-de-investimentos\\fundo_investimento\\squadra\\'

# Caminho completo de todos os arquivos que estão na pasta
paths_arquivos_squadra = glob.glob(path_squadra + '*.xlsx')

lst_dfs = []
for path in paths_arquivos_squadra:
    df = pd.read_excel(path, sheet_name='acoes')
    lst_dfs.append(df)

# Concatendo os dfs de ações
portfolio_squadra = pd.concat(lst_dfs, ignore_index=True)

# Retirando as ações que não tem nenhum dinheiro alocado e os NAN
portfolio_squadra = portfolio_squadra[portfolio_squadra['VL_MERC_POS_FINAL'] != 0]
portfolio_squadra = portfolio_squadra.dropna()

# Definindo a coluna 'data' como o index
portfolio_squadra = portfolio_squadra.set_index('data')
portfolio_squadra

,DENOM_SOCIAL,CD_ATIVO,PORCENTAGEM,VL_MERC_POS_FINAL
data,,,,
2023-01-01,SQUADRA MASTER LONG-BIASED FUNDO DE INVESTIMEN...,EQTL3,2.475635e-01,4.684867e+08
2023-01-01,SQUADRA MASTER LONG-BIASED FUNDO DE INVESTIMEN...,PRIO3,1.011559e-01,1.914264e+08
2023-01-01,SQUADRA MASTER LONG-BIASED FUNDO DE INVESTIMEN...,RAIL3,9.825210e-02,1.859313e+08
2023-01-01,SQUADRA MASTER LONG-BIASED FUNDO DE INVESTIMEN...,GMAT3,8.629957e-02,1.633124e+08
2023-01-01,SQUADRA MASTER LONG-BIASED FUNDO DE INVESTIMEN...,GGPS3,7.809600e-02,1.477881e+08
...,...,...,...,...
2025-02-01,SQUADRA MASTER LONG-BIASED FUNDO DE INVESTIMEN...,CPLE3,7.234322e-03,1.465252e+07
2025-02-01,SQUADRA MASTER LONG-BIASED FUNDO DE INVESTIMEN...,MATD3,6.510763e-03,1.318701e+07
2025-02-01,SQUADRA MASTER LONG-BIASED FUNDO DE INVESTIMEN...,FIEI3,5.004982e-04,1.013718e+06


In [40]:
# Selecionando o conjunto de ações (portfólio) de cada mês - retornando um df com vários dicionários
portfolio_mensal = portfolio_squadra.groupby('data')['CD_ATIVO'].apply(set)

# Convertendo o GroupBy resultante em DataFrame para facilitar a iteração
portfolio_mensal = portfolio_mensal.reset_index()

# Comparando o portfólio de cada mês
comparar_portfolios(df=portfolio_mensal, nome_fundo='Squadra')

Comparando o portfólio de 01/2023 e 02/2023:
O Squadra vendeu as ações: set()
O Squadra comprou as ações: {'INTB3', 'MGLU3', 'VIIA3', 'PETZ3'}
--------------------------------------------------------------------------------
Comparando o portfólio de 02/2023 e 03/2023:
O Squadra vendeu as ações: {'ENEV3', 'PETZ3', 'B3SA3'}
O Squadra comprou as ações: {'WEGE3'}
--------------------------------------------------------------------------------
Comparando o portfólio de 03/2023 e 04/2023:
O Squadra vendeu as ações: {'LWSA3'}
O Squadra comprou as ações: {'CXSE3', 'PETZ3'}
--------------------------------------------------------------------------------
Comparando o portfólio de 04/2023 e 05/2023:
O Squadra vendeu as ações: {'MULT3', 'CXSE3', 'CRFB3', 'ITUB4'}
O Squadra comprou as ações: {'RDOR3'}
--------------------------------------------------------------------------------
Comparando o portfólio de 05/2023 e 06/2023:
O Squadra vendeu as ações: {'CRDE3'}
O Squadra comprou as ações: {'ASAI3',

In [41]:
# Número total de ações de cada mês do portfólio
num_total_acoes_squadra = num_total_acoes(df=portfolio_squadra)
num_total_acoes_squadra

data
2023-01-01    21
2023-02-01    25
2023-03-01    23
2023-04-01    24
2023-05-01    21
2023-06-01    23
2023-07-01    23
2023-08-01    27
2023-09-01    26
2023-10-01    26
2023-11-01    25
2023-12-01    24
2024-01-01    27
2024-02-01    24
2024-03-01    21
2024-04-01    21
2024-05-01    20
2024-06-01    17
2024-07-01    18
2024-08-01    17
2024-09-01    18
2024-10-01    18
2024-11-01    19
2024-12-01    21
2025-01-01    19
2025-02-01    19
Name: CD_ATIVO, dtype: int64

In [42]:
# Rank das 5 maiores ações do fundo
rank_top_5_squadra = rank_top_5(df=portfolio_squadra)
rank_top_5_squadra

data
2023-01-01    [EQTL3, PRIO3, RAIL3, GMAT3, GGPS3]
2023-02-01    [EQTL3, RAIL3, PRIO3, GMAT3, GGPS3]
2023-03-01    [EQTL3, RAIL3, PRIO3, GMAT3, GGPS3]
2023-04-01    [EQTL3, RAIL3, PRIO3, GMAT3, GGPS3]
2023-05-01    [EQTL3, RAIL3, GMAT3, PRIO3, GGPS3]
2023-06-01    [EQTL3, RAIL3, PRIO3, GMAT3, GGPS3]
2023-07-01    [EQTL3, PRIO3, RAIL3, GGPS3, GMAT3]
2023-08-01    [RAIL3, EQTL3, GGPS3, GMAT3, PRIO3]
2023-09-01    [RAIL3, EQTL3, GGPS3, GMAT3, UGPA3]
2023-10-01    [RAIL3, EQTL3, GGPS3, VBBR3, GMAT3]
2023-11-01    [RAIL3, EQTL3, VBBR3, GGPS3, HAPV3]
2023-12-01    [RAIL3, EQTL3, GGPS3, PRIO3, UGPA3]
2024-01-01    [RAIL3, EQTL3, UGPA3, GGPS3, PRIO3]
2024-02-01    [RAIL3, EQTL3, PRIO3, UGPA3, GGPS3]
2024-03-01    [PRIO3, RAIL3, UGPA3, GMAT3, EQTL3]
2024-04-01    [PRIO3, EQTL3, UGPA3, RAIL3, GMAT3]
2024-05-01    [PRIO3, EQTL3, UGPA3, RAIL3, GMAT3]
2024-06-01    [EQTL3, PRIO3, UGPA3, RAIL3, HAPV3]
2024-07-01    [EQTL3, PRIO3, RAIL3, UGPA3, GGPS3]
2024-08-01    [EQTL3, UGPA3, PRIO3, RAIL3, GG

In [ ]:
# Plotando o portfólio de cada mês
plot_portfolio(df=portfolio_squadra, nome_fundo='Squadra')

In [44]:
# Portfólio de ações vendidas do fundo Squadra
lst_dfs = []
for path in paths_arquivos_squadra:
    df = pd.read_excel(path, sheet_name='vendido_acoes')
    lst_dfs.append(df)

# Concatendo os dfs de ações vendidas
portfolio_venda_squadra = pd.concat(lst_dfs, ignore_index=True)

# Retirando as ações vendidas que não tem nenhum dinheiro alocado e os NAN
portfolio_venda_squadra = portfolio_venda_squadra[portfolio_venda_squadra['VL_MERC_POS_FINAL'] != 0]
portfolio_venda_squadra = portfolio_venda_squadra.dropna()

# Definindo a coluna 'data' como o index
portfolio_venda_squadra = portfolio_venda_squadra.set_index('data')
portfolio_venda_squadra

,DENOM_SOCIAL,CD_ATIVO,PORCENTAGEM,VL_MERC_POS_FINAL
data,,,,
2023-01-01,SQUADRA MASTER LONG-BIASED FUNDO DE INVESTIMEN...,ABEV3,0.826328,63719132.66
2023-01-01,SQUADRA MASTER LONG-BIASED FUNDO DE INVESTIMEN...,CRFB3,0.157878,12174118.60
2023-01-01,SQUADRA MASTER LONG-BIASED FUNDO DE INVESTIMEN...,LWSA3,0.015794,1217900.00
2023-02-01,SQUADRA MASTER LONG-BIASED FUNDO DE INVESTIMEN...,MGLU3,0.295537,45448804.06
2023-02-01,SQUADRA MASTER LONG-BIASED FUNDO DE INVESTIMEN...,ABEV3,0.259606,39923316.03
...,...,...,...,...
2025-01-01,SQUADRA MASTER LONG-BIASED FUNDO DE INVESTIMEN...,BRFS3,0.771992,79638468.69
2025-01-01,SQUADRA MASTER LONG-BIASED FUNDO DE INVESTIMEN...,BBDC4,0.183657,18945985.11
2025-01-01,SQUADRA MASTER LONG-BIASED FUNDO DE INVESTIMEN...,WEGE3,0.044351,4575200.00


In [45]:
# Selecionando o conjunto de ações vendidas (portfólio) de cada mês - retornando um df com vários dicionários
portfolio_venda_mensal = portfolio_venda_squadra.groupby('data')['CD_ATIVO'].apply(set)

# Convertendo o GroupBy resultante em DataFrame para facilitar a iteração
portfolio_venda_mensal = portfolio_venda_mensal.reset_index()
portfolio_venda_mensal

,data,CD_ATIVO
0,2023-01-01,"{CRFB3, LWSA3, ABEV3}"
1,2023-02-01,"{INTB3, LWSA3, CRFB3, MGLU3, PETZ3, ABEV3, VIIA3}"
2,2023-03-01,"{INTB3, LWSA3, CRFB3, WEGE3, MGLU3, PETZ3, ABE..."
3,2023-04-01,"{INTB3, CRFB3, WEGE3, MGLU3, PETZ3, ABEV3, VII..."
4,2023-05-01,"{INTB3, RDOR3, WEGE3, MGLU3, PETZ3, ABEV3, VII..."
5,2023-06-01,"{INTB3, LWSA3, ASAI3, RDOR3, WEGE3, MGLU3, PET..."
6,2023-07-01,"{INTB3, LWSA3, ASAI3, RDOR3, WEGE3, MGLU3, PET..."
7,2023-08-01,"{INTB3, LWSA3, BBDC4, RDOR3, WEGE3, MGLU3, PET..."
8,2023-09-01,"{INTB3, BBDC4, RDOR3, WEGE3, MGLU3, IRBR3, PET..."
9,2023-10-01,"{BBDC4, RDOR3, WEGE3, MGLU3, IRBR3, PETZ3, ABE..."


In [46]:
# Número total de ações vendidas de cada mês do portfólio
num_total_acoes_vendidas_squadra = num_total_acoes(df=portfolio_venda_squadra)
num_total_acoes_vendidas_squadra

data
2023-01-01     3
2023-02-01     7
2023-03-01     8
2023-04-01     8
2023-05-01     8
2023-06-01    10
2023-07-01    10
2023-08-01    11
2023-09-01     9
2023-10-01     8
2023-11-01     9
2023-12-01    10
2024-01-01    10
2024-02-01     8
2024-03-01     7
2024-04-01     7
2024-05-01     5
2024-06-01     2
2024-07-01     1
2024-08-01     3
2024-09-01     3
2024-10-01     2
2024-11-01     4
2024-12-01     3
2025-01-01     3
2025-02-01     2
Name: CD_ATIVO, dtype: int64

#### Dfs das opções (posições titulares e lançadas)

In [ ]:
# Ano 2019
df_cda_4_2019 = open_cda_4_v2(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//2019//cda_fi_BLC_4_2019.parquet'
)

# Ano 2020
df_cda_4_2020 = open_cda_4_v2(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//2020//cda_fi_BLC_4_2020.parquet'
)

In [ ]:
# Opções de 2019
df_opcoes_compradas_2019 = fundo_cnpj_opcoes(df=df_cda_4_2019, cnpj='09.412.648/0001-40')[0]
df_opcoes_vendidas_2019 = fundo_cnpj_opcoes(df=df_cda_4_2019, cnpj='09.412.648/0001-40')[1]

# Opções de 2020
df_opcoes_compradas_2020 = fundo_cnpj_opcoes(df=df_cda_4_2020, cnpj='09.412.648/0001-40')[0]
df_opcoes_vendidas_2020 = fundo_cnpj_opcoes(df=df_cda_4_2020, cnpj='09.412.648/0001-40')[1]

In [ ]:
# Transformando os dfs em csv
df_opcoes_compradas_2019.to_csv('C://Users//vitor//projetos_python//python_b3//derivativos//opcoes//arquivos-historicos//opcoes_compradas_2019.csv', sep=';')
df_opcoes_vendidas_2019.to_csv('C://Users//vitor//projetos_python//python_b3//derivativos//opcoes//arquivos-historicos//opcoes_vendidas_2019.csv', sep=';')
df_opcoes_compradas_2020.to_csv('C://Users//vitor//projetos_python//python_b3//derivativos//opcoes//arquivos-historicos//opcoes_compradas_2020.csv', sep=';')
df_opcoes_vendidas_2020.to_csv('C://Users//vitor//projetos_python//python_b3//derivativos//opcoes//arquivos-historicos//opcoes_vendidas_2020.csv', sep=';')

### Fundo Guepardo

* https://maisretorno.com/fundo/guepardo-institucional-master-fia
* Em 05/2025, as maiores posições do fundo são UGPA3, KLBN11 e ALOS3.

In [47]:
# Rentabilidade mensal
df_ret_mensal_guepardo = rentabilidade_fundo(df=df_inf_diario_final, cnpj='14.213.077/0001-54', nome_fundo='guepardo')[0]

# Comparação de rentabilidade entre fundo e benchmark
df_ret_mensal_guepardo_ibov = rentabilidade_fundo_benchmark(
    df_fundo=df_ret_mensal_guepardo,
    df_benchmark=df_ret_ibov_mensal,
    nome_fundo='guepardo',
    nome_benchmark='ibov'
)

df_ret_mensal_guepardo_ibov

,ret_guepardo,ret_ibov,performance
2023-01,-0.74,3.46,-4.20
2023-02,-3.55,-7.49,3.94
2023-03,-0.86,-2.91,2.05
2023-04,6.05,2.50,3.55
2023-05,14.29,3.74,10.55
2023-06,9.84,9.00,0.84
2023-07,5.92,3.26,2.66
2023-08,-2.77,-5.09,2.32
2023-09,1.84,0.71,1.13
2023-10,-3.34,-2.93,-0.41


In [48]:
# Rentabilidade anual
df_ret_anual_guepardo = rentabilidade_fundo(df=df_inf_diario_final, cnpj='14.213.077/0001-54', nome_fundo='guepardo')[1]

# Comparação de rentabilidade entre fundo e benchmark
df_ret_anual_guepardo_ibov = rentabilidade_fundo_benchmark(
    df_fundo=df_ret_anual_guepardo,
    df_benchmark=df_ret_ibov_anual,
    nome_fundo='guepardo',
    nome_benchmark='ibov'
)

df_ret_anual_guepardo_ibov

,ret_guepardo,ret_ibov,performance
2023,49.26,22.28,26.98
2024,-9.00,-10.36,1.36
2025,8.89,17.94,-9.05


In [49]:
guepardo_acoes = fundo_cnpj_acoes(df=df_ativos, cnpj='14.213.077/0001-54')
guepardo_pl = pl_fundo(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_PL_{ano}{mes}.parquet', 
    cnpj='14.213.077/0001-54'
)

# Quantos porcertos cada categoria representa do PL final
pct_acoes = round(pd.Series(guepardo_acoes['VL_MERC_POS_FINAL'].sum() / guepardo_pl.values), 4) 

# Criando a data do portfólio
data = pd.to_datetime(f'{mes}-{ano}')

# Adicionando a informação da porcentagem de cada categoria nos dfs
# Ações
linha_pct_acoes = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_acoes}
df_linha_pct_acoes = pd.DataFrame(linha_pct_acoes)
guepardo_acoes = pd.concat([guepardo_acoes, df_linha_pct_acoes], ignore_index=True)
# Criando a coluna 'data' no df
guepardo_acoes['data'] = data
# Definindo a coluna 'data' como o index
guepardo_acoes = guepardo_acoes.set_index('data')

In [50]:
# O fundo Guepardo é um fundo 100% em ações, tem muito pouco dinheiro em outras categorias
# É o fundo mais "puro" comparado com os outros fundos analisados

# Caminho da pasta do fundo Guepardo
path_guepardo = 'C:\\Users\\vitor\\projetos_python\\python_b3\\composicao-fundos-de-investimentos\\fundo_investimento\\guepardo\\'

# Caminho completo de todos os arquivos que estão na pasta
paths_arquivos_guepardo = glob.glob(path_guepardo + '*.xlsx')

lst_dfs = []
for path in paths_arquivos_guepardo:
    df = pd.read_excel(path, sheet_name='acoes')
    lst_dfs.append(df)

# Concatendo os dfs de ações
portfolio_guepardo = pd.concat(lst_dfs, ignore_index=True)

# Datas dos arquivos excel
data_excel = []
for path in paths_arquivos_guepardo:
    match = re.search(r'(\d{6})', path)
    if match:
        data_excel.append(match.group(1))

# Filtro da porcentagem da alocação de ações do fundo
filt = portfolio_guepardo['DENOM_SOCIAL'] == 'PL% FUNDO'

# Porcentagem da alocação de ações do fundo
pct_acoes_guepardo = portfolio_guepardo.loc[filt, 'VL_MERC_POS_FINAL']

# Transformando em um df da porcentagem da alocação de ações do fundo
df_pct_acoes_guepardo = pd.DataFrame(pct_acoes_guepardo)

In [51]:
# Criando o arquivo excel em que cada categoria está em uma aba diferente
with pd.ExcelWriter(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//fundo_investimento//guepardo//fundo_guepardo_{ano}{mes}.xlsx') as writer:
    guepardo_acoes.to_excel(writer, sheet_name='acoes')

In [52]:
# O fundo Guepardo é um fundo 100% em ações, tem muito pouco dinheiro em outras categorias
# É o fundo mais "puro" comparado com os outros fundos analisados

# Caminho da pasta do fundo Guepardo
path_guepardo = 'C:\\Users\\vitor\\projetos_python\\python_b3\\composicao-fundos-de-investimentos\\fundo_investimento\\guepardo\\'

# Caminho completo de todos os arquivos que estão na pasta
paths_arquivos_guepardo = glob.glob(path_guepardo + '*.xlsx')

lst_dfs = []
for path in paths_arquivos_guepardo:
    df = pd.read_excel(path, sheet_name='acoes')
    lst_dfs.append(df)

# Concatendo os dfs de ações
portfolio_guepardo = pd.concat(lst_dfs, ignore_index=True)

# Datas dos arquivos excel
data_excel = []
for path in paths_arquivos_guepardo:
    match = re.search(r'(\d{6})', path)
    if match:
        data_excel.append(match.group(1))

# Filtro da porcentagem da alocação de ações do fundo
filt = portfolio_guepardo['DENOM_SOCIAL'] == 'PL% FUNDO'

# Porcentagem da alocação de ações do fundo
pct_acoes_guepardo = portfolio_guepardo.loc[filt, 'VL_MERC_POS_FINAL']

# Transformando em um df da porcentagem da alocação de ações do fundo
df_pct_acoes_guepardo = pd.DataFrame(pct_acoes_guepardo)

# Transformando o index do df nas datas dos arquivos excel
df_pct_acoes_guepardo.index = data_excel
df_pct_acoes_guepardo

,VL_MERC_POS_FINAL
202301,0.9392
202302,0.9239
202303,0.9636
202304,0.9678
202305,0.9686
202306,0.9495
202307,0.9532
202308,0.9372
202309,0.9498
202310,0.9769


Períodos que o fundo possuia caixa/outras categorias:
* 10/2012: títulos públicos (LFT - 15,58%) e cotas de fundo (fundos DI - 11,62%);
* 06/2013: títulos públicos (LFT - 5,11%) e cotas de fundo (fundos DI - 11,33%);
* 09/2014: cotas de fundo (fundos DI - 12,29%);
* 02/2015: cotas de fundo (fundos DI - 9,63%);
* 11/2016: títulos públicos (LFT - 13,90%) e cotas de fundo (fundos DI - 27,39%);
* 01/2017: títulos públicos (LFT - 21,26%) e cotas de fundo (fundos DI - 13,61%);
* 02/2018: títulos públicos (LFT - 4,36%) e cotas de fundo (fundos DI - 20,67%);
* 10/2019: cotas de fundo (fundos DI - 19,38%);
* 01/2020: cotas de fundo (fundos DI - 24,62%);
* 2021 - comprado em ações;
* 2022 - comprado em ações;
* 12/2023 - operações compromissadas (NTNB - 9,66%);

In [53]:
# Caminho da pasta do fundo Guepardo
path_guepardo = 'C:\\Users\\vitor\\projetos_python\\python_b3\\composicao-fundos-de-investimentos\\fundo_investimento\\guepardo\\'

# Caminho completo de todos os arquivos que estão na pasta
paths_arquivos_guepardo = glob.glob(path_guepardo + '*.xlsx')

lst_dfs = []
for path in paths_arquivos_guepardo:
    df = pd.read_excel(path, sheet_name='acoes')
    lst_dfs.append(df)

# Concatendo os dfs de ações
portfolio_guepardo = pd.concat(lst_dfs, ignore_index=True)

# Retirando as ações que não tem nenhum dinheiro alocado  e os NAN
portfolio_guepardo = portfolio_guepardo[portfolio_guepardo['VL_MERC_POS_FINAL'] != 0]
portfolio_guepardo = portfolio_guepardo.dropna()

# Definindo a coluna 'data' como o index
portfolio_guepardo = portfolio_guepardo.set_index('data')
portfolio_guepardo

,DENOM_SOCIAL,CD_ATIVO,PORCENTAGEM,VL_MERC_POS_FINAL
data,,,,
2023-01-01,GUEPARDO INSTITUCIONAL MASTER FUNDO DE INVESTI...,VULC3,0.247012,2.492254e+08
2023-01-01,GUEPARDO INSTITUCIONAL MASTER FUNDO DE INVESTI...,KLBN11,0.204068,2.058972e+08
2023-01-01,GUEPARDO INSTITUCIONAL MASTER FUNDO DE INVESTI...,ITUB4,0.137043,1.382715e+08
2023-01-01,GUEPARDO INSTITUCIONAL MASTER FUNDO DE INVESTI...,UGPA3,0.127014,1.281520e+08
2023-01-01,GUEPARDO INSTITUCIONAL MASTER FUNDO DE INVESTI...,RAIL3,0.119408,1.204777e+08
...,...,...,...,...
2025-02-01,GUEPARDO INSTITUCIONAL MASTER FUNDO DE INVESTI...,ITUB4,0.030896,6.159472e+07
2025-02-01,GUEPARDO INSTITUCIONAL MASTER FUNDO DE INVESTI...,CSAN3,0.019208,3.829310e+07
2025-02-01,GUEPARDO INSTITUCIONAL MASTER FUNDO DE INVESTI...,ITUB3,0.013547,2.700839e+07


In [54]:
# Selecionando o conjunto de ações (portfólio) de cada mês - retornando um df com vários dicionários
portfolio_mensal = portfolio_guepardo.groupby('data')['CD_ATIVO'].apply(set)

# Convertendo o GroupBy resultante em DataFrame para facilitar a iteração
portfolio_mensal = portfolio_mensal.reset_index()

# Comparando o portfólio de cada mês
comparar_portfolios(df=portfolio_mensal, nome_fundo='Guepardo')

Comparando o portfólio de 01/2023 e 02/2023:
O Guepardo vendeu as ações: set()
O Guepardo comprou as ações: set()
--------------------------------------------------------------------------------
Comparando o portfólio de 02/2023 e 03/2023:
O Guepardo vendeu as ações: set()
O Guepardo comprou as ações: set()
--------------------------------------------------------------------------------
Comparando o portfólio de 03/2023 e 04/2023:
O Guepardo vendeu as ações: set()
O Guepardo comprou as ações: set()
--------------------------------------------------------------------------------
Comparando o portfólio de 04/2023 e 05/2023:
O Guepardo vendeu as ações: {'DXCO3'}
O Guepardo comprou as ações: {'GGBR4'}
--------------------------------------------------------------------------------
Comparando o portfólio de 05/2023 e 06/2023:
O Guepardo vendeu as ações: {'GMAT3', 'MLAS3'}
O Guepardo comprou as ações: {'FLRY3'}
--------------------------------------------------------------------------------


In [55]:
# Número total de ações de cada mês do portfpolio
num_total_acoes_guepardo = num_total_acoes(df=portfolio_guepardo)
num_total_acoes_guepardo 

data
2023-01-01    12
2023-02-01    12
2023-03-01    12
2023-04-01    12
2023-05-01    12
2023-06-01    11
2023-07-01    12
2023-08-01    12
2023-09-01    11
2023-10-01    13
2023-11-01    13
2023-12-01    13
2024-01-01    16
2024-02-01    16
2024-03-01    16
2024-04-01    18
2024-05-01    17
2024-06-01    17
2024-07-01    17
2024-08-01    16
2024-09-01    16
2024-10-01    13
2024-11-01    13
2024-12-01    13
2025-01-01    13
2025-02-01    13
Name: CD_ATIVO, dtype: int64

In [56]:
# Rank das 5 maiores ações do fundo
rank_top_5_guepardo = rank_top_5(df=portfolio_guepardo)
rank_top_5_guepardo

data
2023-01-01    [VULC3, KLBN11, ITUB4, UGPA3, RAIL3]
2023-02-01    [VULC3, KLBN11, ITUB4, UGPA3, RAIL3]
2023-03-01    [VULC3, KLBN11, UGPA3, ITUB4, RAIL3]
2023-04-01    [VULC3, KLBN11, UGPA3, ITUB4, RAIL3]
2023-05-01    [VULC3, KLBN11, UGPA3, ITUB4, RAIL3]
2023-06-01    [VULC3, UGPA3, KLBN11, ITUB4, RAIL3]
2023-07-01    [VULC3, KLBN11, UGPA3, ITUB4, RAIL3]
2023-08-01    [VULC3, KLBN11, UGPA3, ITUB4, RAIL3]
2023-09-01    [VULC3, KLBN11, UGPA3, ITUB4, RAIL3]
2023-10-01    [VULC3, UGPA3, KLBN11, ITUB4, RAIL3]
2023-11-01    [VULC3, UGPA3, KLBN11, ITUB4, GMAT3]
2023-12-01    [VULC3, KLBN11, UGPA3, ITUB4, GMAT3]
2024-01-01    [VULC3, KLBN11, UGPA3, ITUB4, GMAT3]
2024-02-01    [KLBN11, VULC3, UGPA3, ITUB4, GMAT3]
2024-03-01    [KLBN11, VULC3, UGPA3, ALOS3, ITUB4]
2024-04-01    [KLBN11, ALOS3, VULC3, ITUB4, UGPA3]
2024-05-01    [KLBN11, ALOS3, VULC3, ITUB4, FLRY3]
2024-06-01    [KLBN11, ALOS3, VULC3, ITUB4, FLRY3]
2024-07-01    [KLBN11, ALOS3, VULC3, FLRY3, UGPA3]
2024-08-01    [KLBN11, ALO

In [ ]:
# Plotando o portfólio de cada mês
plot_portfolio(df=portfolio_guepardo, nome_fundo='Guepardo')

### Comparação dos fundos

In [57]:
# Comparação da rentabilidade mensal
lst_ret_mensal = [
    df_ret_mensal_verde, 
    df_ret_mensal_dynamo,
    df_ret_mensal_ip,
    df_ret_mensal_squadra,
    df_ret_mensal_guepardo
]

# Concatenando os dfs
df_ret_mensal = pd.concat(lst_ret_mensal, axis=1)
df_ret_mensal

,ret_verde,ret_dynamo,ret_ip,ret_squadra,ret_guepardo
DT_COMPTC,,,,,
2023-01,0.65,4.80,5.11,5.23,-0.74
2023-02,-4.23,-4.66,-5.15,-6.62,-3.55
2023-03,-1.75,-6.02,0.56,-0.20,-0.86
2023-04,-1.48,-0.24,3.51,3.92,6.05
2023-05,5.05,12.20,3.03,8.83,14.29
2023-06,7.85,8.40,7.54,8.13,9.84
2023-07,2.09,5.00,4.21,4.00,5.92
2023-08,-2.25,-6.35,-0.94,2.24,-2.77
2023-09,-0.18,-0.77,-1.87,1.13,1.84


In [58]:
# Comparação da rentabilidade anual
lst_ret_anual = [
    df_ret_anual_verde, 
    df_ret_anual_dynamo,
    df_ret_anual_ip,
    df_ret_anual_squadra,
    df_ret_anual_guepardo
]

# Concatenando os dfs
df_ret_anual = pd.concat(lst_ret_anual, axis=1)
df_ret_anual

,ret_verde,ret_dynamo,ret_ip,ret_squadra,ret_guepardo
DT_COMPTC,,,,,
2023,11.57,24.92,30.94,44.86,49.26
2024,-14.93,-9.84,7.27,-19.27,-9.00
2025,20.79,21.95,19.41,29.60,8.89


In [59]:
# Comparação da performance dos fundos
lst_performance_fundo_ibov = [
    df_ret_mensal_verde_ibov['performance'],
    df_ret_mensal_dynamo_ibov['performance'],
    df_ret_mensal_ip_ibov['performance'],
    df_ret_mensal_squadra_ibov['performance'],
    df_ret_mensal_guepardo_ibov['performance'],
]

# Concatenando os dfs
df_performance_fundo_ibov = pd.concat(lst_performance_fundo_ibov, axis=1)
df_performance_fundo_ibov.columns = ['verde', 'dynamo', 'ip', 'squadra', 'guepardo']
df_performance_fundo_ibov = df_performance_fundo_ibov.dropna()
df_performance_fundo_ibov

,verde,dynamo,ip,squadra,guepardo
2023-01,-2.81,1.34,1.65,1.77,-4.20
2023-02,3.26,2.83,2.34,0.87,3.94
2023-03,1.16,-3.11,3.47,2.71,2.05
2023-04,-3.98,-2.74,1.01,1.42,3.55
2023-05,1.31,8.46,-0.71,5.09,10.55
2023-06,-1.15,-0.60,-1.46,-0.87,0.84
2023-07,-1.17,1.74,0.95,0.74,2.66
2023-08,2.84,-1.26,4.15,7.33,2.32
2023-09,-0.89,-1.48,-2.58,0.42,1.13
2023-10,-3.08,-1.97,-0.76,2.24,-0.41


In [60]:
# Contando quantos meses da perfomance dos fundos foram positivos e negativos
resultados = {}
for col in df_performance_fundo_ibov.columns:
    positivos = (df_performance_fundo_ibov[col] > 0).sum()
    negativos = (df_performance_fundo_ibov[col] < 0).sum()
    resultados[col] = {'Positivos': positivos, 'Negativos': negativos}

for coluna, valores in resultados.items():
    print(f"Fundo {coluna}:")
    print(f"  Positivos: {valores['Positivos']}")
    print(f"  Negativos: {valores['Negativos']}")
    print('='*15)

Fundo verde:
  Positivos: 14
  Negativos: 15
Fundo dynamo:
  Positivos: 17
  Negativos: 12
Fundo ip:
  Positivos: 18
  Negativos: 11
Fundo squadra:
  Positivos: 17
  Negativos: 12
Fundo guepardo:
  Positivos: 16
  Negativos: 13


In [61]:
# Comparação do número total de ações do portfólio
lst_num_total_acoes = [
    num_total_acoes_verde,
    num_total_acoes_dynamo,
    num_total_acoes_ip,
    num_total_acoes_squadra,
    num_total_acoes_guepardo,
]

# Concatenando os dfs
df_num_total_acoes = pd.concat(lst_num_total_acoes, axis=1)
df_num_total_acoes.columns = ['verde', 'dynamo', 'ip', 'squadra', 'guepardo']
df_num_total_acoes

,verde,dynamo,ip,squadra,guepardo
data,,,,,
2023-01-01,71,30,5,21,12
2023-02-01,70,27,4,25,12
2023-03-01,61,27,4,23,12
2023-04-01,60,28,4,24,12
2023-05-01,59,30,5,21,12
2023-06-01,58,31,7,23,11
2023-07-01,69,31,7,23,12
2023-08-01,51,31,7,27,12
2023-09-01,46,31,7,26,11


In [62]:
# Comparação do rank top 5 do portfólio
lst_rank_top_5 = [
    rank_top_5_verde,
    rank_top_5_dynamo,
    rank_top_5_ip,
    rank_top_5_squadra,
    rank_top_5_guepardo,
]

# Concatenando os dfs
df_rank_top_5 = pd.concat(lst_rank_top_5, axis=1)
df_rank_top_5.columns = ['verde', 'dynamo', 'ip', 'squadra', 'guepardo']
df_rank_top_5

,verde,dynamo,ip,squadra,guepardo
data,,,,,
2023-01-01,"[SUZB3, RENT3, EQTL3, VBBR3, RAIL3]","[ENEV3, VBBR3, CSAN3, RENT3, NTCO3]","[ITSA4, EQTL3, HAPV3, MULT3, RDOR3]","[EQTL3, PRIO3, RAIL3, GMAT3, GGPS3]","[VULC3, KLBN11, ITUB4, UGPA3, RAIL3]"
2023-02-01,"[SUZB3, RENT3, RAIL3, VBBR3, EQTL3]","[ENEV3, VBBR3, CSAN3, NTCO3, RDOR3]","[EQTL3, ITSA4, RDOR3, HAPV3]","[EQTL3, RAIL3, PRIO3, GMAT3, GGPS3]","[VULC3, KLBN11, ITUB4, UGPA3, RAIL3]"
2023-03-01,"[SUZB3, RAIL3, VBBR3, ASAI3, SBSP3]","[ENEV3, VBBR3, CSAN3, NTCO3, RDOR3]","[EQTL3, ITSA4, ITUB4, MULT3]","[EQTL3, RAIL3, PRIO3, GMAT3, GGPS3]","[VULC3, KLBN11, UGPA3, ITUB4, RAIL3]"
2023-04-01,"[SUZB3, RAIL3, VBBR3, RENT3, EQTL3]","[ENEV3, CSAN3, VBBR3, RDOR3, NTCO3]","[EQTL3, ITSA4, ITUB4, MULT3]","[EQTL3, RAIL3, PRIO3, GMAT3, GGPS3]","[VULC3, KLBN11, UGPA3, ITUB4, RAIL3]"
2023-05-01,"[SUZB3, RAIL3, EQTL3, VBBR3, SBSP3]","[ENEV3, VBBR3, RDOR3, SUZB3, NTCO3]","[EQTL3, ITSA4, ITUB4, VIVA3, MULT3]","[EQTL3, RAIL3, GMAT3, PRIO3, GGPS3]","[VULC3, KLBN11, UGPA3, ITUB4, RAIL3]"
2023-06-01,"[SUZB3, RENT3, RAIL3, EQTL3, VBBR3]","[ENEV3, VBBR3, RDOR3, NTCO3, CSAN3]","[EQTL3, ITSA4, RENT3, VIVA3, ELET3]","[EQTL3, RAIL3, PRIO3, GMAT3, GGPS3]","[VULC3, UGPA3, KLBN11, ITUB4, RAIL3]"
2023-07-01,"[SUZB3, RAIL3, RENT3, VBBR3, ITUB4]","[ENEV3, RDOR3, VBBR3, NTCO3, CSAN3]","[ITSA4, EQTL3, VIVA3, ITUB4, RENT3]","[EQTL3, PRIO3, RAIL3, GGPS3, GMAT3]","[VULC3, KLBN11, UGPA3, ITUB4, RAIL3]"
2023-08-01,"[SUZB3, RAIL3, VBBR3, SBSP3, RENT3]","[ENEV3, VBBR3, RDOR3, NTCO3, CSAN3]","[ITSA4, EQTL3, ITUB4, RENT3, VIVA3]","[RAIL3, EQTL3, GGPS3, GMAT3, PRIO3]","[VULC3, KLBN11, UGPA3, ITUB4, RAIL3]"
2023-09-01,"[SUZB3, RAIL3, SBSP3, VBBR3, RENT3]","[ENEV3, VBBR3, RDOR3, NTCO3, CSAN3]","[ITSA4, RENT3, EQTL3, ELET3, VIVA3]","[RAIL3, EQTL3, GGPS3, GMAT3, UGPA3]","[VULC3, KLBN11, UGPA3, ITUB4, RAIL3]"


## Fundos de Previdência

### Itau Prev High Yield

* https://maisretorno.com/fundo/itau-flexprev-high-yield-ii-fi-financeiro-mult-cp-resp-limitada

In [ ]:
itau_prev_hy_debentures = fundo_cnpj_debentures(df=df_ativos, cnpj='42.814.944/0001-42')
itau_prev_hy_pl = pl_fundo(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_PL_{ano}{mes}.parquet', 
    cnpj='42.814.944/0001-42'
)

# Quantos porcertos cada categoria representa do PL final
pct_debentures = round(pd.Series(itau_prev_hy_debentures['VL_MERC_POS_FINAL'].sum() / itau_prev_hy_pl.values), 4) 

# Criando a data do portfólio
data = pd.to_datetime(f'{mes}-{ano}')

# Adicionando a informação da porcentagem de cada categoria nos dfs
# Debêntures
linha_pct_debentures = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_debentures}
df_linha_pct_debentures = pd.DataFrame(linha_pct_debentures)
itau_prev_hy_debentures = pd.concat([itau_prev_hy_debentures, df_linha_pct_debentures], ignore_index=True)
# Criando a coluna 'data' no df
itau_prev_hy_debentures['data'] = data
# Definindo a coluna 'data' como o index
itau_prev_hy_debentures = itau_prev_hy_debentures.set_index('data')

In [ ]:
# Criando o arquivo excel para o fundo Itau Prev HY
with pd.ExcelWriter(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//fundo_previdencia//itau_hy//fundo_itau_prev_hy_{ano}{mes}.xlsx') as writer:
    itau_prev_hy_debentures.to_excel(writer, sheet_name='debentures')

### Itau Person Kinea CP

* https://maisretorno.com/fundo/kinea-cp-prev-rf-fi

In [ ]:
itau_kinea_prev_debentures = fundo_cnpj_debentures(df=df_ativos, cnpj='26.491.419/0001-87')
itau_kinea_prev_pl = pl_fundo(
    path=f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fundos_cvm//{ano}{mes}//cda_fi_PL_{ano}{mes}.parquet', 
    cnpj='26.491.419/0001-87'
)

# Quantos porcertos cada categoria representa do PL final
pct_debentures = round(pd.Series(itau_kinea_prev_debentures['VL_MERC_POS_FINAL'].sum() / itau_kinea_prev_pl.values), 4) 

# Criando a data do portfólio
data = pd.to_datetime(f'{mes}-{ano}')

# Adicionando a informação da porcentagem de cada categoria nos dfs
# Debêntures
linha_pct_debentures = {'DENOM_SOCIAL': 'PL% FUNDO', 'VL_MERC_POS_FINAL' : pct_debentures}
df_linha_pct_debentures = pd.DataFrame(linha_pct_debentures)
itau_kinea_prev_debentures = pd.concat([itau_kinea_prev_debentures, df_linha_pct_debentures], ignore_index=True)
# Criando a coluna 'data' no df
itau_kinea_prev_debentures['data'] = data
# Definindo a coluna 'data' como o index
itau_kinea_prev_debentures = itau_kinea_prev_debentures.set_index('data')

In [ ]:
# Criando o arquivo excel para o fundo Itau Kinea Prev
with pd.ExcelWriter(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//fundo_previdencia//itau_person_kinea//fundo_itau_person_kinea_{ano}{mes}.xlsx') as writer:
    itau_kinea_prev_debentures.to_excel(writer, sheet_name='debentures')